In [ ]:
# import numpy as np
# from scipy.integrate import solve_ivp
# from copy import deepcopy
# from scipy.spatial.transform import Rotation as ScipyRot
# import matplotlib.pyplot as plt
# import time as systemtime
# import inspect

# from scipy.interpolate import interp1d, CubicSpline

# import plotly.graph_objects as go
# from plotly.subplots import make_subplots


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from copy import deepcopy
from scipy.spatial.transform import Rotation as ScipyRot
import matplotlib.pyplot as plt
import time as systemtime
import pandas as pd
import matplotlib.patches as patches
import inspect
from scipy.interpolate import interp1d, CubicSpline

## SUPPORTING FUNCTION

In [ ]:
# class agent():
#     def __init__(self, **kwargs):
#         self.save_fig = kwargs.get('save_fig', True)
#         self.need_y_lim =  kwargs.get('need_y_lim', True)
#         self.fig_name = kwargs.get('fig_name', "")

#         # Motion
#         self.p_formula =  kwargs.get('p_formula', "")
#         self.omega_formula =  kwargs.get('omega_formula', "")
#         self.v_formula =  kwargs.get('v_formula', "")

#         # Leader and landmarks
#         self.z = kwargs.get('z', "")
#         self.z_act = kwargs.get('z_act', "")
#         self.is_z_moving =  kwargs.get('is_z_moving', "")

#         # Helper parameters
#         self.agent_name = kwargs.get("agent_name", "")
#         self.time = kwargs.get('time', (0, 100))
#         self.tol = kwargs.get("tol", 1e-3)
#         self.dt = kwargs.get("dt", 0.0001)
#         self.current_time = 0 
#         self.numStaticLandmark = kwargs.get("numStaticLandmark", "")
#         self.HAVE_PRINTED = False
#         self.z_label = kwargs.get("z_label", "")

#         # Plotting lists
#         self.plot_act_p = []
#         self.plot_est_p = []
#         self.plot_err_lambda_bar = []
#         self.plot_est_lambda_bar = []
#         self.plot_act_lambda_bar = []
#         self.plot_z = []
#         self.correction_term = {}
#         for label in self.z_label:
#             self.correction_term[label] = [[0.0,0.0,0.0,0.0,0.0,0.0]]
#         self.current_correction = []
#         # Communication delay
#         self.delay = kwargs.get("delay", {"Landmark": -1, "Leader": -1, "Follower": -1})

#         # Noise
#         self.noise = kwargs.get("noise", {"linear":  lambda: np.array([0.0, 0.0, 0.0]), 
#                                           "angular": lambda: np.array([0.0, 0.0, 0.0]),
#                                           "SourcePoint": lambda: 0})

#         # Tuning parameters
#         self.k= kwargs.get("k", 1)
#         self.q= kwargs.get("q", 10)
#         self.v= kwargs.get("v", [0.1, 1])
#         self.l= kwargs.get("l", "")
#         self.Q=np.diag(np.hstack([np.diag(self.q*np.eye(3)) for i in range(self.l)]))
#         self.V = np.diag(np.hstack([np.diag(self.v[i]*np.eye(3)) for i in range(len(self.v))]))
#         self.p_ricatti = kwargs.get("p_ricatti", [1, 10])
#         self.P_ricatti = np.diag(np.hstack([np.diag(self.p_ricatti[i]*np.eye(3)) for i in range(len(self.p_ricatti))]))
        
#         # initial estimation
#         self.Lambda_hat_0 = kwargs.get("Lambda_hat_0", np.array([np.sqrt(3)/2, 0.5, 0, 0]).T)  # quaternion: w, x, y, z
#         self.Lambda_0 = kwargs.get("Lambda_0", np.array([1, 0, 0, 0]).T)  # quaternion: w, x, y, z
#         self.p_hat = kwargs.get("p_hat", np.array([3,2,7], dtype=np.float64).T)
#         self.Rot = np.eye(3)
#         self.Rot_hat = np.matmul(self.Rot, self.rodrigues_formula(self.Lambda_0)) 
#         self.p_hat = self.add_bar(self.Rot_hat, self.p_hat)


#         initial_state = np.concatenate((self.Lambda_0.flatten(), self.Lambda_hat_0.flatten(), self.p_hat.flatten(), self.P_ricatti.flatten()))
#         self.soly = []
#         self.solt = []
#         self.solnumlandmark = []
#         self.solimage = []
#         self.caltime = []
#         self.soly.append(initial_state)

#     def function_S(self, input):
#         '''
#         Create a 3x3 skew-symmetric matrix, S(x)y = x x y
#         Input: 3x1 array
#         Output: 3x3 array
#         '''
#         # input should be array
#         # output array
#         flattened_input = input.flatten()
#         output = [[0,           -flattened_input[2],    flattened_input[1]],
#                 [flattened_input[2],  0,              -flattened_input[0]],
#                 [-flattened_input[1], flattened_input[0],     0]]
#         return np.array(output)

#     def rodrigues_formula(self, quaternion):
#         '''
#         Quaternion -> R_tilde_bar
#         Input: [w,x,y,z]
#         Output R_tile_bar (rotation matrix)
#         From page6
#         '''
#         return np.eye(3) + 2*np.matmul(self.function_S(quaternion[1:]), (quaternion[0]*np.eye(3) + self.function_S(quaternion[1:])))
    
#     def function_Pi(self, input):
#         '''
#         Pi_x := I_3 - xx^T
#         Input: array
#         Output P_x
#         '''
#         return np.eye(3) - np.outer(input, input)
    
#     def add_bar(self, input_rot, input_p):
#         '''
#         Change frame (F -> B)
#         '''
#         return np.matmul(np.transpose(input_rot), input_p)
    
#     def function_g(self, input_rot_i, input_p_ij):
#         '''
#         Calculate direction g_ij := R_i^T p_ij/||p_ij||
#         Input:
#             Rotation matrix R: 3x3 array
#             pose p: 3x1 array
#             landmark z : 3x1 array
#             with_noise : boolean
#         Output: 
#             direction vector 3x1 array
#         '''
#         norm = input_p_ij/np.linalg.norm(input_p_ij)
#         dir = np.matmul(np.transpose(input_rot_i), norm)
#         # if self.noise["SourcePoint"]() == 1:
#         #     '''
#         #     calculate noisy d = sign(d_{i,3}) / demon (d_{i,1}/d_{i,3} + n_{n,1}, d_{i,2}/d_{i,3} + n_{i,2}, 1).T
#         #     demon = sqrt((d_{i,1}/d_{i,3} + n_{n,1})^2 + (d_{i,2}/d_{i,3} + n_{i,2})^2 + 1)
#         #     '''
#         #     dir = dir.flatten()
#         #     n_1 = np.random.uniform(-0.001, 0.001, 1)[0]
#         #     n_2 = np.random.uniform(-0.001, 0.001, 1)[0]
#         #     d1_d3 = dir[0]/dir[2] + n_1
#         #     d2_d3 = dir[1]/dir[2] + n_2
#         #     demon = np.sqrt(d1_d3**2 + d2_d3**2 + 1)
#         #     dir = (np.sign(dir[2])/ demon) * np.array([[d1_d3, d2_d3, 1]]).T
#         return dir
    
#     def function_A(self, omega):
#         '''
#         Create the A maxtrix 
#         Input = 3x1 array
#         Output = 6x6 matrix
#         '''
#         A11 = self.function_S(-omega)
#         A12 = np.zeros((3,3))
#         A21 = np.zeros((3,3))
#         A22 = self.function_S(-omega)
#         return np.vstack((np.hstack((A11, A12)), np.hstack((A21, A22))))

#     def function_C(self, input_R_i, input_R_hat_i, input_p_ij, input_p_hat_ij, current_z):
#         '''
#         Create the C maxtrix 
#         Input = ...
#         Output = num_landmark*3x6 matrix
#         '''
#         for j in range(self.l):
#             # pi_g
#             pi_g = self.function_Pi(self.function_g(input_R_i, input_p_ij[j]))
#             second = self.function_S(np.matmul(np.transpose(input_R_hat_i), current_z[j]))
#             # Pi(g_ij) S(epsolon) hat_R_i^T
#             # original paper (first)
#             # first = np.matmul(pi_g, np.matmul(function_S(np.matmul(np.transpose(input_R_hat_i), input_p_hat_ij[j])), np.transpose(input_R_hat_i)))
#             # distributed pose localization (first)
#             final = -np.matmul(pi_g, second)
#             C_landmark = np.hstack((final, pi_g))
#             if j == 0:
#                 output_C = C_landmark
#             else:
#                 output_C = np.vstack((output_C, C_landmark))

#         return output_C
    
#     def observer_equations(self, input_omega, input_vel, input_R_hat_i, input_p_hat_ij, input_R_i, input_p_ij, input_P, input_p_bar_hat, current_z):
#         first_upper = input_omega
#         first_lower = -np.matmul(self.function_S(input_omega), np.transpose(input_p_bar_hat)[0]).reshape((3,1)) + self.add_bar(input_R_i, input_vel)
#         first_part = np.vstack((first_upper, first_lower))
#         # second
#         final = np.transpose(np.array([0, 0, 0], dtype=np.float64))
#         final2 = np.transpose(np.array([0, 0, 0], dtype=np.float64))
#         self.current_correction = []
#         for j in range(self.l):
#             # pi_g_ij
#             pi_g = self.function_Pi(self.function_g(input_R_i, input_p_ij[j]))
#             # pi_g_ij *(p_bar_hat - R_hat_i.T * p_hat_ij)
#             common = np.matmul(pi_g, (np.transpose(input_p_bar_hat).reshape((3,)) - np.matmul(np.transpose(input_R_hat_i), current_z[j])))
#             # S(R_hat_i.T * p_hat_j)
#             S_rp = self.function_S(np.matmul(np.transpose(input_R_hat_i), current_z[j]))
#             # q* S() * pi_g * R_hat_i.T *(p_bar_hat - R_hat_i.T * p_hat_ij)
#             upper = self.q * np.matmul(S_rp, common)
#             # q* R_hat_i * pi_g * R_hat_i.T * p_hat_ij
#             lower = self.q * common
#             self.current_correction.append(np.concatenate([upper,lower]).tolist())
#             final += upper
#             final2 += lower
#         second_part = np.vstack((final, final2)).reshape((6,1))

#         second_part = self.k * np.matmul(input_P, second_part)
#         output_omega_hat_p_hat_dot = first_part - second_part
#         return output_omega_hat_p_hat_dot
    
#     def function_p_ij(self, p, z, p_delay):
#         all_p_ij = []
#         for j in range(self.l):
#             if not "Landmark" in self.z_label:
#                 pij = (np.transpose(p) - (z[j]))[0].tolist()
#             else:
#                 pij = (np.transpose(p_delay) - (z[j]))[0].tolist()
#             all_p_ij.append(pij)
#         return np.array(all_p_ij)
    
#     def dynamics(self, t, y):
#         # , input_k, input_z, input_q, input_Q, input_V, numNeighbor, is_z_moving, p_formula, omega_formula, v_formula
#         qua_flat, qua_hat_flat, p_hat_flat, input_P_flat = np.split(y, [4, 8, 11])
#         qua_flat = qua_flat/np.linalg.norm(qua_flat)
#         qua_hat_flat = qua_hat_flat/np.linalg.norm(qua_hat_flat)
#         input_R = self.rodrigues_formula(qua_flat)
#         input_R_hat = self.rodrigues_formula(qua_hat_flat)

#         input_p_hat = p_hat_flat.reshape((3,1))
#         input_P = input_P_flat.reshape((6,6))

#         input_p = self.p_formula(t)
#         input_omega = self.omega_formula(t) + self.noise['angular']().reshape((3,1))
#         input_vel = self.v_formula(t) + self.noise['linear']().reshape((3,1))

#         if self.is_z_moving:
#             if not all(value < 0 for value in self.delay.values()):
#                 current_z = self.z(t, self.delay["Leader"], self.delay["Follower"])
#                 delayed_z = self.z(t, self.delay["Leader"], self.delay["Follower"]) ##consider delay from agents and leader only
#             else:
#                 current_z = self.z(t, 0, 0)
#                 current_z_act = self.z_act(t, 0, 0)
#         else:
#             current_z = self.z
#             current_z_act = self.z_act
            
#         # if not all(value < 0 for value in self.delay.values()):
#         #     # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#         #     input_delayed_p = self.p_formula(max(t, t-self.delay["Landmark"]))
#         #     input_p_ij = self.function_p_ij(input_p, current_z, input_delayed_p) #!!!!!!!!!! delay for landmark from detection
#         #     # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#         #     input_p_hat_ij = self.function_p_ij(input_p_hat, delayed_z, input_p_hat) #!!!!! here
#         # else:
#         input_p_ij = self.function_p_ij(input_p, current_z_act, input_p)
#         input_p_hat_ij = self.function_p_ij(input_p_hat, current_z, input_p_hat)

#         if 10<t<10.01 or 40<t<40.01 or 70<t<70.01 or 95<t<95.005:
#             if not self.HAVE_PRINTED: 
#                 print("current_z", current_z)
#                 self.HAVE_PRINTED = True
#         else:
#             self.HAVE_PRINTED = False
#         input_A = self.function_A(input_omega)

#         input_C = self.function_C(input_R, input_R_hat, input_p_ij, input_p_hat_ij, current_z)
#         ####################################

#         ####################################
#         ############# Observer #############
#         output_omega_hat_p_hat_dot = self.observer_equations(input_omega, input_vel, input_R_hat, input_p_hat_ij, input_R, input_p_ij, input_P, input_p_hat, current_z)
#         ############# Observer #############
#         ####################################
#         output_P_dot = np.matmul(input_A, input_P) + np.matmul(input_P, np.transpose(input_A)) - np.matmul(input_P, np.matmul(np.transpose(input_C), np.matmul(self.Q, np.matmul(input_C, input_P)))) + self.V
#         omega_hat = output_omega_hat_p_hat_dot[0:3]
#         omega_hat = omega_hat.reshape((-1,))
#         p_hat_dot = output_omega_hat_p_hat_dot[3:]
#         ####################################
#         ############ Quaternion ############
#         omega_hat_4x4 = np.array([[0, -omega_hat[0], -omega_hat[1], -omega_hat[2]],
#                                 [omega_hat[0], 0, omega_hat[2], -omega_hat[1]],
#                                 [omega_hat[1], -omega_hat[2], 0, omega_hat[0]],
#                                 [omega_hat[2], omega_hat[1], -omega_hat[0], 0]])

#         output_qua_hat_flat = 0.5*np.matmul(omega_hat_4x4, qua_hat_flat)
#         input_omega = input_omega.flatten()
#         omega_4x4 = np.array([[0, -input_omega[0], -input_omega[1], -input_omega[2]],
#                                 [input_omega[0], 0, input_omega[2], -input_omega[1]],
#                                 [input_omega[1], -input_omega[2], 0, input_omega[0]],
#                                 [input_omega[2], input_omega[1], -input_omega[0], 0]])
#         output_qua_flat = 0.5*np.matmul(omega_4x4, qua_flat)
#         return np.concatenate((output_qua_flat, output_qua_hat_flat, p_hat_dot.flatten(), output_P_dot.flatten()))
#         ########### Quaternion ############
#         ####################################

#     def rk45_step(self):
#         # , t, y, dt, args, tol
#         a2, a3, a4, a5, a6 = 1/5, 3/10, 4/5, 8/9, 1
#         b21 = 1/5
#         b31, b32 = 3/40, 9/40
#         b41, b42, b43 = 44/45, -56/15, 32/9
#         b51, b52, b53, b54 = 19372/6561, -25360/2187, 64448/6561, -212/729
#         b61, b62, b63, b64, b65 = 9017/3168, -355/33, 46732/5247, 49/176, -5103/18656
#         c1, c2, c3, c4, c5, c6 = 35/384, 0, 500/1113, 125/192, -2187/6784, 11/84

#         c1_4, c3_4, c4_4, c5_4, c6_4 = 5179/57600, 7571/16695, 393/640, -92097/339200, 187/2100
#         # Runge-Kutta stages
#         k1 = self.dynamics(self.current_time, self.soly[-1])
#         k2 = self.dynamics(self.current_time + a2*self.dt, self.soly[-1] + self.dt*b21*k1)
#         k3 = self.dynamics(self.current_time + a3*self.dt, self.soly[-1] + self.dt*(b31*k1 + b32*k2))
#         k4 = self.dynamics(self.current_time + a4*self.dt, self.soly[-1] + self.dt*(b41*k1 + b42*k2 + b43*k3))
#         k5 = self.dynamics(self.current_time + a5*self.dt, self.soly[-1] + self.dt*(b51*k1 + b52*k2 + b53*k3 + b54*k4))
#         k6 = self.dynamics(self.current_time + a6*self.dt, self.soly[-1] + self.dt*(b61*k1 + b62*k2 + b63*k3 + b64*k4 + b65*k5))

#         # Update step
#         y_next = self.soly[-1] + self.dt*(c1*k1 + c2*k2 + c3*k3 + c4*k4 + c5*k5 + c6*k6)

#         y_next_4 = self.soly[-1] + self.dt * (c1_4*k1 + c3_4*k3 + c4_4*k4 + c5_4*k5 + c6_4*k6)

#         error = np.abs(y_next - y_next_4)
#         error_norm = np.linalg.norm(error)
#         safety_factor = 0.9
#         min_scale_factor = 0.2
#         max_scale_factor = 40.0
#         if error_norm <= self.tol:
#             success = True
#             t = self.current_time + self.dt
#             dt = self.dt * min(max_scale_factor, max(min_scale_factor, safety_factor * (self.tol / error_norm)**0.25))
#         else:
#             success = False
#             t = self.current_time
#             dt = self.dt * max(min_scale_factor, safety_factor * (self.tol / error_norm)**0.25)
#         return y_next, t, dt, success
    
#     def solver(self):
#         #k, z, q, Q, V, l, is_z_moving, p_formula, omega_formula, v_formula, tol, dt, time, soly
#         ### Run solver
#         ######################################################
#         ####################### Solver #######################
#         start_time = systemtime.time()
#         run_time = systemtime.time()
#         while self.current_time <= self.time[1]:
#             simulation_time_str = f"Simulation time: {self.current_time}"
#             # print(f"{simulation_time_str}{' ' * (40 - len(simulation_time_str))}Run time: {systemtime.time()-run_time}", end='\r', flush=True)
#             y_next, next_time, new_dt, success = self.rk45_step()
#             if success:
#                 for i, label in enumerate(self.z_label):
#                     self.correction_term[label].append(self.current_correction[i])
#                 self.current_correction = []
#                 self.soly.append(y_next)
#                 self.solt.append(self.current_time)
#                 self.solimage.append([self.current_time, self.l])
#                 self.solnumlandmark.append(self.l)
#                 end_time = systemtime.time()
#                 self.caltime.append(end_time - start_time)
#                 start_time = end_time
#                 break
#                 if next_time >= self.time[1]:
#                     self.solt.append(self.time[1])
#                     break
#                 self.dt = new_dt
#                 self.current_time = next_time
#             else:
#                 self.dt = new_dt

#         args = (self.k, self.z, self.q, self.Q, self.V, self.l, self.is_z_moving, self.p_formula, self.omega_formula, self.v_formula)
#         ####################### Solver #######################
#         ######################################################
#         self.extract_info_for_plotting()
#         return self.solt, self.soly, self.caltime, args
    
#     def extract_info_for_plotting(self):
#         self.plot_est_p = []
#         for t, solution in zip(self.solt, self.soly):
#             p_act = self.p_formula(t).flatten().tolist()
#             self.plot_act_p.append(p_act)
#             if self.l > 1:
#                 if self.is_z_moving:
#                     self.plot_z.append(self.z(t,self.delay["Leader"], self.delay["Follower"]).tolist())
#                 else:
#                     self.plot_z.append(self.z)
#             else:
#                 if self.is_z_moving:
#                     self.plot_z.append(self.z(t,self.delay["Leader"], self.delay["Follower"]).tolist())
#                 else:
#                     self.plot_z.append(self.z)
#             sol_R = self.rodrigues_formula(solution[:4])
#             sol_est_R = self.rodrigues_formula(solution[4:8])
#             self.plot_est_p.append(np.matmul(sol_est_R, np.array(solution[8:11])))
#             err = np.matmul(np.array(sol_R).reshape((3,3)), np.transpose(np.array(sol_est_R).reshape((3,3))))
#             err = ScipyRot.from_matrix(err)
#             est = ScipyRot.from_matrix(np.array(sol_est_R).reshape((3,3)))
#             act = ScipyRot.from_matrix(np.array(sol_R).reshape((3,3)))
#             self.plot_err_lambda_bar.append(err.as_quat().tolist())
#             self.plot_est_lambda_bar.append(est.as_quat().tolist())
#             self.plot_act_lambda_bar.append(act.as_quat().tolist())
#         try:
#                 z_string = str(inspect.getsourcelines(self.z)[0]).strip("['\\n']").split(" = ")[1]
#         except:
#             z_string = str(self.z)

#         noise_temp_str = {key: "".join(inspect.getsourcelines(value)[0][0]).strip().split(':')[2] for key, value in self.noise.items()}
#         noise_str = ""
#         for key, value in noise_temp_str.items():
#             noise_str += f' {key}: {value}\n'

#     def plotting(self, y_lim_correction=None, library="matplotlib"):
#         trace_to_plot1 = [np.array(self.plot_act_p)-np.array(self.plot_est_p),
#                           np.array(self.plot_err_lambda_bar)[:,0:3]]
#         for label in self.z_label:
#             trace_to_plot1.append(np.array(self.correction_term[label])[:,3:].tolist())
#             trace_to_plot1.append(np.array(self.correction_term[label])[:,:3].tolist())

#         trace_to_plot2 = [np.array(self.plot_act_p)-np.array(self.plot_est_p),
#                           np.array(self.plot_err_lambda_bar)[:,0:3], 
#                           np.array(self.plot_act_p), 
#                           np.array(self.plot_est_p)]
#         for idx in range(len(self.plot_z[0])):
#             trace_to_plot2.append(np.array(self.plot_z)[:,idx,:])

#         labels = ["x", "y", "z"]
#         colors = ["royalblue", "mediumseagreen", "tomato"]

#         if library == "matplotlib":
#             %matplotlib widget
#             figure, ax = plt.subplots(int(np.ceil((len(trace_to_plot1)+1)/2)),2, figsize=(15,int(np.ceil((len(trace_to_plot1)+1)/2))*5))
#             titles = ['Position error', 'Orientation error in F frame']
#             y_limits = [0.5, 0.1]
#             for label in self.z_label:
#                 titles.append("Position correction from " + label)
#                 y_limits.append(y_lim_correction)
#                 titles.append("Orientation correction from " + label)
#                 y_limits.append(y_lim_correction)

#             for i in range(len(trace_to_plot1)):
#                 row = int(i/2)
#                 col = i%2
#                 ax[row,col].plot(self.solt, trace_to_plot1[i], marker='o', markersize=0.1)
#                 ax[row,col].set_xlabel(titles[i])
#                 ax[row,col].grid()
#                 ax[row,col].set_xlim(0)
#                 if self.need_y_lim and not y_limits[i] == None: 
#                     ax[row,col].set_ylim(-y_limits[i], y_limits[i])
#                 ax[row,col].minorticks_on()
#                 for k,j in enumerate(ax[row,col].lines):
#                     j.set_color(colors[k])
#                     j.set_label(labels[k])
#                     j.set_linewidth(0.3)
#                 ax[row,col].legend(loc="upper right")
         
#             try:
#                 z_string = str(inspect.getsourcelines(self.z)[0]).strip("['\\n']").split(" = ")[1]
#             except:
#                 z_string = str(self.z)

#             noise_temp_str = {key: "".join(inspect.getsourcelines(value)[0][0]).strip().split(':')[2] for key, value in self.noise.items()}
#             noise_str = ""
#             for key, value in noise_temp_str.items():
#                 noise_str += f' {key}: {value}\n'
                
#             param = {'p_formula': str(inspect.getsourcelines(self.p_formula)[0]).strip("['\\n']").split(" = ")[1],
#                     'omega_formula': str(inspect.getsourcelines(self.omega_formula)[0]).strip("['\\n']").split(" = ")[1],
#                     'vel_formula':str(inspect.getsourcelines(self.v_formula)[0]).strip("['\\n']").split(" = ")[1],
#                     'z': z_string[:int(len(z_string)/2)] + "\n" + z_string[int(len(z_string)/2):],
#                     'is_z_moving': self.is_z_moving,
#                     'k': self.k,
#                     'q': self.q,
#                     'l': self.l,
#                     'v': self.v,
#                     'p_ricatti': self.p_ricatti,
#                     'tol': self.tol,
#                     'Lambda_hat_0': self.Lambda_hat_0,
#                     'p_hat': self.p_hat,
#                     'delay': self.delay, 
#                     'noise': noise_str}

#             param_text = "\n".join([f"{k} = {v}" for k, v in param.items()])
#             row = int((i+1)/2)
#             col = (i+1) % 2
#             ax[row,col].xaxis.set_visible(False)
#             ax[row,col].yaxis.set_visible(False)
#             ax[row,col].set_frame_on(False)
#             ax[row,col].text(0.5, 0.5, param_text, fontsize=10, va='center', ha='center', ma='center',
#                 bbox=dict(boxstyle="round", alpha=0.1))
                
#             figure.suptitle(self.fig_name + " " + self.agent_name)
#             if self.save_fig:
#                 plt.savefig(FOLDERPATH+ self.fig_name +   "_" + str(TRIAL) + "_" + self.agent_name + "_2d.png")

#             ############################################################################################################
#             %matplotlib widget
#             figure, ax = plt.subplots(int(np.ceil((len(trace_to_plot2)+1)/2)),2, figsize=(15,int(np.ceil((len(trace_to_plot2)+1)/2))*5))
#             titles = ['Position error', 'Orientation error', "Actual position", "Estimated position"]
#             for label in self.z_label:
#                 titles.append("Position of " + label)

#             for i in range(len(trace_to_plot2)):
#                 row = int(i/2)
#                 col = i%2
#                 ax[row,col].plot(self.solt, trace_to_plot2[i], marker='o', markersize=0.1)
#                 ax[row,col].set_xlabel(titles[i])
#                 ax[row,col].grid()
#                 ax[row,col].set_xlim(0)
#                 ax[row,col].minorticks_on()
#                 for k,j in enumerate(ax[row,col].lines):
#                     j.set_color(colors[k])
#                     j.set_label(labels[k])
#                     j.set_linewidth(0.3)
#                 ax[row,col].legend(loc="upper right")

#             row = int((i+1)/2)
#             col = (i+1) % 2
#             ax[row,col].xaxis.set_visible(False)
#             ax[row,col].yaxis.set_visible(False)
#             ax[row,col].set_frame_on(False)
#             ax[row,col].text(0.5, 0.5, param_text, fontsize=10, va='center', ha='center', ma='center',
#                 bbox=dict(boxstyle="round", alpha=0.1))
            
#             figure.suptitle(self.fig_name + " " + self.agent_name)
#             figure.show()
#             if self.save_fig:
#                 plt.savefig(FOLDERPATH+ self.fig_name +   "_" + str(TRIAL) + "_" + self.agent_name + "_debug_2d.png")

#         elif library == "plotly":
#             pass

#     def plotting_3d(self, library="plotly"):
#         if library=="matplotlib":
#             %matplotlib widget
#             ax = plt.figure(figsize=(15,15)).add_subplot(projection='3d')

#             ax.plot(np.array(self.plot_est_p)[:,0], np.array(self.plot_est_p)[:,1], np.array(self.plot_est_p)[:,2], label='Estimated pose for ' + self.agent_name)
#             ax.plot(np.array(self.plot_est_p)[0,0], np.array(self.plot_est_p)[0,1], np.array(self.plot_est_p)[0,2], ms=5, marker="o") # , label='Estimtaed poes for ' + self.agent_name
#             ax.plot(np.array(self.plot_act_p)[:,0], np.array(self.plot_act_p)[:,1], np.array(self.plot_act_p)[:,2], label='Actual pose for ' + self.agent_name, linestyle="dotted")
#             ax.plot(np.array(self.plot_act_p)[0,0], np.array(self.plot_act_p)[0,1], np.array(self.plot_act_p)[0,2], ms=5, marker="o") # , label='Actual pose start for ' + self.agent_name
#             for idx in range(self.l):
#                 if "Leader" in self.z_label[idx]:
#                     # Plotting leader
#                     ax.plot(np.array(self.plot_z)[:,idx,0], np.array(self.plot_z)[:,idx,1], np.array(self.plot_z)[:,idx,2], label=self.z_label[idx])
#                     ax.plot(np.array(self.plot_z)[0,idx,0], np.array(self.plot_z)[0,idx,1], np.array(self.plot_z)[0,idx,2], ms=5, marker="o") # , label='Leader start'
#                 if "Landmark" in self.z_label[idx]:
#                     # Plotting static landmark
#                     ax.plot(np.array(self.plot_z)[:,idx,0], np.array(self.plot_z)[:,idx,1], np.array(self.plot_z)[:,idx,2], label=self.z_label[idx], ms=5, marker="o")
#                 if "Follower" in self.z_label[idx]:
#                     # Plotting moving follower
#                     ax.plot(np.array(self.plot_z)[:,idx, 0], np.array(self.plot_z)[:,idx, 1], np.array(self.plot_z)[:,idx, 2], label=self.z_label[idx])
#                     ax.plot(np.array(self.plot_z)[0,idx, 0], np.array(self.plot_z)[0,idx, 1], np.array(self.plot_z)[0,idx, 2], ms=5, marker="o") # , label='Moving agent i='+str(idx)+" start"

#             ax.legend()
#             ax.set_xticks([-4,-2,0,2,4,6,8,10,12])
#             ax.set_yticks([-4,-2,0,2,4,6,8,10,12])
#             ax.set_zticks([0,2,4,6,8])
#             plt.show()
#             if self.save_fig:
#                 plt.savefig(FOLDERPATH+ self.fig_name +   "_" + str(TRIAL) + "_" + self.agent_name + "_3d.png")
#         elif library == "plotly":
#             traceList = []
            
#             colors = ['orange','green','purple','brown','pink','gray','yellow','red','magenta','lime','teal','lavender','maroon','navy','olive','silver','skyblue','indigo']

#             traceList.append(go.Scatter3d(
#                     x=np.array(self.plot_est_p)[:,0], y=np.array(self.plot_est_p)[:,1], z=np.array(self.plot_est_p)[:,2], 
#                     mode='lines', line=dict(color='blue', width=1.5), name='Estimated pose for ' + self.agent_name))

#             traceList.append(go.Scatter3d(
#                     x=[np.array(self.plot_est_p)[0,0]], y=[np.array(self.plot_est_p)[0,1]], z=[np.array(self.plot_est_p)[0,2]],
#                     mode='markers', marker=dict(size=5, color='blue'), showlegend=False))

#             traceList.append(go.Scatter3d(
#                     x=np.array(self.plot_act_p)[:,0], y=np.array(self.plot_act_p)[:,1], z=np.array(self.plot_act_p)[:,2],
#                     mode='lines', line=dict(color='cyan', width=1.5), name='Actual pose for ' + self.agent_name))
        
#             traceList.append(go.Scatter3d(
#                     x=[np.array(self.plot_act_p)[0,0]], y=[np.array(self.plot_act_p)[0,1]], z=[np.array(self.plot_act_p)[0,2]],
#                     mode='markers', marker=dict(size=5, color='cyan'), showlegend=False))
#             for idx in range(self.l):
#                 if not "Landmark" in self.z_label[idx]:
#                     traceList.append(go.Scatter3d(
#                         x=np.array(self.plot_z)[:,idx,0], y=np.array(self.plot_z)[:,idx,1], z=np.array(self.plot_z)[:,idx,2],
#                         mode='lines', line=dict(color=colors[idx], width=1.5), name=self.z_label[idx]))
#                     traceList.append(go.Scatter3d(
#                         x=[np.array(self.plot_z)[0,idx,0]], y=[np.array(self.plot_z)[0,idx,1]], z=[np.array(self.plot_z)[0,idx,2]],
#                         mode='markers', marker=dict(color=colors[idx], size=5), showlegend=False))
#                 elif "Landmark" in self.z_label[idx]:
#                     traceList.append(go.Scatter3d(
#                         x=[np.array(self.plot_z)[0,idx,0]], y=[np.array(self.plot_z)[0,idx,1]], z=[np.array(self.plot_z)[0,idx,2]],
#                         mode='markers', marker=dict(color=colors[idx], size=5), name=self.z_label[idx]))

#             fig = go.Figure(data=traceList)

#             fig.update_layout(
#                 title=self.fig_name + " " + self.agent_name,
#                 scene=dict(xaxis=dict(title='X', range=[-12, 12]), yaxis=dict(title='Y', range=[-12, 12]), zaxis=dict(title='Z', range=[-12, 12]) ),
#                 width=1000, height=1000,

#             )

#             fig.show()
#             if self.save_fig:
#                 fig.write_html(FOLDERPATH+ self.fig_name +  "_" + str(TRIAL) + "_" + self.agent_name + "_3d.html")
#                 # fig.write_image(FOLDERPATH+ self.fig_name + "_" + str(TRIAL) + "_" + self.agent_name + "_3d.png")

In [ ]:
class riccati_observer():
    def __init__(self, **kwargs):
        ######################################################
        ##################### Parameters #####################

        # for key, value in kwargs.items():
            # setattr(self, key, value)

        self.use_adaptive = kwargs.get('use_adaptive', True)
        self.quaternion = kwargs.get('quaternion', True)
        self.noise = kwargs.get('noise', False)
        self.which_eq = kwargs.get('which_eq', 0)
        self.with_image_hz_sim = kwargs.get('with_image_hz_sim', False)
        self.randomize_image_input = kwargs.get('randomize_image_input', False)
        self.which_omega = kwargs.get('which_omega','full') # "z" or "full"
        self.time = kwargs.get('time',(0, 10))
        self.stepsize = kwargs.get('stepsize', 0.1)

        self.image_hz = kwargs.get('image_hz', 20)
        self.image_time = np.arange(self.time[0], self.time[1]+1/self.image_hz, 1/self.image_hz)
        self.tol = kwargs.get('tol', 1e-2 * 3) 

        # tol = 1e-2 * 5
        # tol = 0.05

        self.name = kwargs.get("name", "")
        ##################### Parameters #####################
        ######################################################

        ######################################################
        ################### initialization ###################
        # landmarks
        self.z_appear = kwargs.get('z_appear', "")
        self.z_appear_act = kwargs.get('z_appear_act', "")
        self.k = kwargs.get('k', 1)
        self.v = kwargs.get('v', [0.1,1])
        self.l = kwargs.get('l', 3)
        self.q = kwargs.get('q', 10)
        self.q = self.q
        self.V = np.diag(np.hstack([np.diag(self.v[i]*np.eye(3)) for i in range(len(self.v))]))
        self.Q = np.diag(np.hstack([np.diag(self.q*np.eye(3)) for i in range(self.l)]))

        self.Rot = np.eye(3)
        ## R_hat = R x R_tilde_bar
        self.Lambda_bar_0 = kwargs.get('Lambda_bar_0', np.array([np.sqrt(2)/2, np.sqrt(2)/2, 0, 0]).T)  # quaternion: w, x, y, z
        self.Lambda_0 = np.array([1, 0, 0, 0]).T  # quaternion: w, x, y, z

        self.Rot_hat = np.matmul(self.Rot, self.rodrigues_formula(self.Lambda_bar_0)) 
        self.p_hat = kwargs.get('p_hat', np.array([[-2, 4, 3]], dtype=np.float64).T)

        self.p_formula =  kwargs.get('p_formula', "")
        self.omega_formula =  kwargs.get('omega_formula', "")
        self.v_formula =  kwargs.get('v_formula', "")

        self.p_bar_hat = self.add_bar(self.Rot_hat, self.p_hat)
        self.p_ricatti = kwargs.get('p_ricatti', [1,100])
        self.P_ricatti = np.diag(np.hstack([np.diag(self.p_ricatti[i]*np.eye(3)) for i in range(len(self.p_ricatti))]))
        
        if self.quaternion:
            initial_state = np.concatenate((self.Lambda_0.flatten(), self.Lambda_bar_0.flatten(), self.p_bar_hat.flatten(), self.P_ricatti.flatten()))
        else:
            initial_state = np.concatenate((self.Rot.flatten(), self.Rot_hat.flatten(), self.p_bar_hat.flatten(), self.P_ricatti.flatten()))

        self.soly = []
        self.solt = []
        self.solimage = []
        self.solnumlandmark = []
        self.caltime = []
        self.soly.append(initial_state)
        self.current_time = 0
        self.dt = self.stepsize

        self.show_measurement = True
        self.i = 0
        self.v_linear = 0.6 # m/s
        self.omega_z = 0.2  # rad/s
        self.turn_duration = np.pi / 2 / self.omega_z
        ################### initialization ###################
        ######################################################

        self.print_init()

    def print_init(self):
        Q_str = '   \n'.join(['                             ' + '  '.join(map(str, row)) for row in self.Q])
        V_str = '   \n'.join(['                             ' + '  '.join(map(str, row)) for row in self.V])
        P_ricatti_str = '   \n'.join(['                             ' + '  '.join(map(str, row)) for row in self.P_ricatti])

        print_p_formula = str(inspect.getsourcelines(self.p_formula)[0]).strip("['\\n']").split(" = ")[1]
        print_v_formula = str(inspect.getsourcelines(self.v_formula)[0]).strip("['\\n']").split(" = ")[1]
        print_omega_formula = str(inspect.getsourcelines(self.omega_formula)[0]).strip("['\\n']").split(" = ")[1]
        print_z = str(inspect.getsourcelines(self.z_appear)[0]).strip("['\\n']").split(" = ")[1]
        print(f"""
        Parameters
        p_formula              | {print_p_formula}
        v_linear               | {print_v_formula}
        omega_z                | {print_omega_formula}
        use_adaptive           | {self.use_adaptive}
        quaternion             | {self.quaternion}
        time                   | {self.time}
        stepsize               | {self.stepsize}
        tol                    | {self.tol}
        noise                  | {self.noise}
        number of landmark     | {self.l}
        Initial estimate pose  | {self.p_hat.flatten()}
        Initial estimate ori   | {self.Lambda_bar_0.flatten()}
        k                      | {self.k}
        z_appear               | {print_z}
        q                      | {self.q}
        V                      | {self.v}
        P ricatti              | {self.p_ricatti}
        """)

    def function_S(self, input):
        '''
        Create a 3x3 skew-symmetric matrix, S(x)y = x x y
        Input: 3x1 array
        Output: 3x3 array
        '''
        # input should be array
        # output array
        flattened_input = input.flatten()
        output = [[0,           -flattened_input[2],    flattened_input[1]],
                [flattened_input[2],  0,              -flattened_input[0]],
                [-flattened_input[1], flattened_input[0],     0]]
        return np.array(output)

    def rodrigues_formula(self, quaternion):
        '''
        Quaternion -> R_tilde_bar
        Input: [w,x,y,z]
        Output R_tile_bar (rotation matrix)
        From page6
        '''
        return np.eye(3) + 2*np.matmul(self.function_S(quaternion[1:]), (quaternion[0]*np.eye(3) + self.function_S(quaternion[1:])))

    def function_A(self, omega):
        '''
        Create the A maxtrix 
        Input = 3x1 array
        Output = 6x6 matrix
        '''
        A11 = self.function_S(-omega)
        A12 = np.zeros((3,3))
        A21 = np.zeros((3,3))
        A22 = self.function_S(-omega)
        return np.vstack((np.hstack((A11, A12)), np.hstack((A21, A22))))

    def function_Pi(self, input):
        '''
        Pi_x := I_3 - xx^T
        Input: array
        Output P_x
        '''
        # hmm: changed to outer: NO diff
        return np.eye(3) - np.matmul(input, np.transpose(input))
        # return np.eye(3) - np.outer(input, input)

    # hmm: not using d in input simulation: got the first measurement from the dummy input/ simulation. (only ran the first step to check) same in static condition, diff in linear (prob caz at different time step)
    def function_d(self, input_rot, input_p, input_z, with_noise):
        '''
        Calculate direction d_i(t) := R^T(t)(p(t) - z_i)/|p(t)-z_i|
        Input:
            Rotation matrix R: 3x3 array
            pose p: 3x1 array
            landmark z : 3x1 array
            with_noise : boolean
        Output: 
            direction vector 3x1 array
        '''
        norm = (input_p - input_z)/np.linalg.norm(input_p - input_z)
        dir = np.matmul(np.transpose(input_rot), norm)
        return dir

    def function_C(self, input_R, input_R_hat, input_p, input_z, input_z_act, with_noise, num_landmarks):
        '''
        Create the C maxtrix 
        Input = ...
        Output = num_landmark*3x6 matrix
        '''
        for landmark_idx in range(num_landmarks):
            # S(R_hat.T x z)
            # hmm: d calculation is different: same as above
            first = self.function_Pi(self.function_d(input_R, input_p, np.transpose(input_z_act[landmark_idx]), with_noise))
            second = self.function_S(np.matmul(np.transpose(input_R_hat), np.transpose(input_z[landmark_idx])))
            final = -np.matmul(first, second)
            C_landmark = np.hstack((final, first))
            if landmark_idx == 0:
                output_C = C_landmark
            else:
                output_C = np.vstack((output_C, C_landmark))
        return output_C

    def add_bar(self, input_rot, input_p):
        '''
        Change frame (F -> B)
        '''
        return np.matmul(np.transpose(input_rot), input_p)

    def remove_bar(self, input_rot, input_p_bar):
        '''
        Change frame (B -> F)
        '''
        return np.matmul(np.linalg.inv(np.transpose(input_rot)), input_p_bar)

    def GetlinVelocity(self, t):
        return self.v_formula(t)

    def GetAngVelocity(self, t):
        return self.omega_formula(t)

    def GetPose(self, t):
        return self.p_formula(t)
    
    def GetLandmark(self, t):
        return self.z_appear(t)
    
    def GetLandmark_act(self, t):
        return self.z_appear_act(t)
    
    def visual_plot(self, figsize = (20, 4), bound_y=True):
        '''
        Plot the result of estimation
        '''
        name = str(self.which_eq) + "_" + str(self.which_omega)

        plot_est_p_bar = []
        plot_act_p_bar = []
        plot_est_p = []
        plot_act_p = []
        # for t, solution in zip(self.solt, self.soly):
        #     if self.quaternion:
        #         own_sol_R = self.rodrigues_formula(solution[:4])
        #         own_sol_est_R = self.rodrigues_formula(solution[4:8])
        #         own_sol_est_p_bar = solution[8:11]
        #     else:
        #         own_sol_R = solution[:9]
        #         own_sol_est_R = solution[9:18]
        #         own_sol_est_p_bar = solution[18:21]
        #     p = self.GetPose(t).reshape((3,))
        #     # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
        #     p_bar_temp = np.matmul(np.transpose(np.array(own_sol_R).reshape((3,3))), p)
        #     p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)

        #     plot_est_p_bar.append(own_sol_est_p_bar.tolist())
        #     plot_act_p_bar.append(p_bar_temp.tolist())
        #     plot_est_p.append(p_temp.tolist())
        #     plot_act_p.append(p.tolist())
        #     x_coords_temp = [coord[0] for coord in plot_est_p]
        #     y_coords_temp = [coord[1] for coord in plot_est_p]
        #     x_coords = [coord[0] for coord in plot_act_p]
        #     y_coords = [coord[1] for coord in plot_act_p]

        scipy_plot_est_p = []
        scipy_plot_act_p = []

        # for t, rotation, p_bar_hat in zip(sol_t, sol_R, sol_est_p_bar):
        #     p = self.GetPose(t).reshape((3,))
        #     # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
        #     rotation = self.rodrigues_formula(rotation)
        #     p_temp = np.matmul(np.transpose(np.array(rotation).reshape((3,3))), p)
        #     scipy_plot_est_p.append(p_bar_hat.tolist())
        #     scipy_plot_act_p.append(p_temp.tolist())
    ###############################################################################################################################
    ###############################################################################################################################
        plot_err_lambda_bar = []
        plot_act_lambda_bar = []
        plot_est_lambda_bar = []
        plt_err_degree = []
        for idx, (t, solution) in enumerate(zip(self.solt, self.soly)):
            if self.quaternion:
                own_sol_R = self.rodrigues_formula(solution[:4])
                own_sol_est_R = self.rodrigues_formula(solution[4:8])
                own_sol_est_p_bar = solution[8:11]
            else:
                own_sol_R = solution[:9]
                own_sol_est_R = solution[9:18]
                own_sol_est_p_bar = solution[18:21]
            p = self.GetPose(t).reshape((3,))
            # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
            p_bar_temp = np.matmul(np.transpose(np.array(own_sol_R).reshape((3,3))), p)
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)

            plot_est_p_bar.append(own_sol_est_p_bar.tolist())
            plot_act_p_bar.append(p_bar_temp.tolist())
            plot_est_p.append(p_temp.tolist())
            plot_act_p.append(p.tolist())

            err = np.matmul(np.linalg.inv(np.array(own_sol_est_R).reshape((3,3))), np.array(own_sol_R).reshape((3,3)))
            est = ScipyRot.from_matrix(np.array(own_sol_est_R).reshape((3,3)))
            act = ScipyRot.from_matrix(np.array(own_sol_R).reshape((3,3)))
            err = ScipyRot.from_matrix(err)
            err_q = err.as_quat().tolist()
            est_q = est.as_quat().tolist()
            act_q = act.as_quat().tolist()
            plot_err_lambda_bar.append(err_q)
            plot_est_lambda_bar.append(est_q)
            plot_act_lambda_bar.append(act_q)
            err_degree = err.as_euler('xyz', degrees=True)
            plt_err_degree.append(err_degree)
            
        figure, ax = plt.subplots(1,2, figsize=(22,5))
        colorlist = ["tomato", "limegreen", "slateblue"]
        print("hiii")
        try:
            # ax[0,0].plot(self.solt, np.array(plot_act_p_bar)-np.array(plot_est_p_bar), label=["new x", "new y", "new z"], marker='o', markersize=0.2)
            # # ax[0,0].plot(sol_t, np.array(scipy_plot_act_p)-scipy_plot_est_p, label=["standard x", "standard y", "standard z"], linestyle='dotted', marker='o', markersize=0.2)
            # ax[0,0].legend(loc="upper right")
            # ax[0,0].set_xlabel("pose error in B frame")
            # ax[0,0].grid()
            # ax[0,0].set_xlim(0)
            # if bound_y:
            #     ax[0,0].set_ylim(-0.5, 0.5)
            # ax[0,0].minorticks_on()
            if "1" in self.name:
                name = r"Vehicle $f_1$"
                ylabel = [r'$\tilde{p}_{f_1}$ (m)' , r'$\tilde{\theta}_{f_1}$ (deg)']
            elif "2" in self.name:
                name = r"Vehicle $f_2$"
                ylabel = [r'$\tilde{p}_{f_2}$ (m)' , r'$\tilde{\theta}_{f_2}$ (deg)']
            elif "3" in self.name:
                name = r"Vehicle $f_3$"
                ylabel = [r'$\tilde{p}_{f_3}$ (m)' , r'$\tilde{\theta}_{f_3}$ (deg)']
            elif "4" in self.name:
                name = r"Vehicle $f_4$"
                ylabel = [r'$\tilde{p}_{f_4}$ (m)' , r'$\tilde{\theta}_{f_4}$ (deg)']
            elif "5" in self.name:
                name = r"Vehicle $f_5$"
                ylabel = [r'$\tilde{p}_{f_5}$ (m)' , r'$\tilde{\theta}_{f_5}$ (deg)']

            err_pose = np.array(plot_act_p)-np.array(plot_est_p)
            ax[0].plot(self.solt, err_pose[:,0], label="x error", linewidth=3, color=colorlist[0])
            ax[0].plot(self.solt, err_pose[:,1], label="y error", linewidth=3, color=colorlist[1])
            ax[0].plot(self.solt, err_pose[:,2], label="z error", linewidth=3, color=colorlist[2])
            ax[0].legend(markerscale=15, loc="center left", fontsize=24, bbox_to_anchor=(0.65, 0.85))
            ax[0].set_xlabel("Time (sec)",  fontsize=24)
            ax[0].set_ylabel(ylabel[0],     fontsize=24)
            ax[0].grid()
            ax[0].set_xlim(0)
            ax[0].set_ylim(-0.5, 0.5)
            ax[0].minorticks_on()
            
            ax[0].tick_params(axis='both', labelsize=20)
            ax[1].tick_params(axis='both', labelsize=20)
            # ax[0,1].plot(self.solt, np.array(plot_err_lambda_bar)[:,0:3], label=["x", "y", "z"], marker='o', markersize=0.2, color=colorlist)
            ax[1].plot(self.solt, np.array(plt_err_degree)[:,0], label="roll error", linewidth=3, color=colorlist[0])
            ax[1].plot(self.solt, np.array(plt_err_degree)[:,1], label="pitch error", linewidth=3, color=colorlist[1])
            ax[1].plot(self.solt, np.array(plt_err_degree)[:,2], label="yaw error", linewidth=3, color=colorlist[2])
            ax[1].legend(markerscale=15, loc="center left", fontsize=24, bbox_to_anchor=(0.65, 0.9))
            ax[1].set_xlabel("Time (sec)",  fontsize=24)
            ax[1].set_ylabel(ylabel[1],     fontsize=24)
            ax[1].grid()
            ax[1].set_xlim(0)
            ax[1].set_ylim(-100, 100)
            ax[1].minorticks_on()
            
            figure.suptitle(f"{name}", fontsize=32)

            figure, ax = plt.subplots(3,2, figsize=figsize)
            err_pose = np.array(plot_act_p)-np.array(plot_est_p)
            ax[0,0].plot(self.solt, err_pose[:,0], label="x error", marker='o', markersize=0.2, color=colorlist[0])
            ax[0,0].plot(self.solt, err_pose[:,1], label="y error", marker='o', markersize=0.2, color=colorlist[1])
            ax[0,0].plot(self.solt, err_pose[:,2], label="z error", marker='o', markersize=0.2, color=colorlist[2])
            ax[0,0].legend(loc="upper right", fontsize=18)
            ax[0,0].set_xlabel("Time (sec)", fontsize=18)
            ax[0,0].grid()
            ax[0,0].set_xlim(0)
            if bound_y:
                ax[0,0].set_ylim(-0.5, 0.5)
            ax[0,0].minorticks_on()

            # ax[0,1].plot(self.solt, np.array(plot_err_lambda_bar)[:,0:3], label=["x", "y", "z"], marker='o', markersize=0.2, color=colorlist)
            ax[0,1].plot(self.solt, np.array(plt_err_degree)[:,0], label="roll error", marker='o', markersize=0.2, color=colorlist[0])
            ax[0,1].plot(self.solt, np.array(plt_err_degree)[:,1], label="pitch error", marker='o', markersize=0.2, color=colorlist[1])
            ax[0,1].plot(self.solt, np.array(plt_err_degree)[:,2], label="yaw error", marker='o', markersize=0.2, color=colorlist[2])
            ax[0,1].legend(loc="upper right", fontsize=18)
            ax[0,1].set_xlabel("Time (sec)", fontsize=18)
            ax[0,1].grid()
            ax[0,1].set_xlim(0)
            ax[0,1].set_ylim(-180, 180)
            ax[0,1].minorticks_on()

            ax[1,0].plot(self.solt, np.array(plot_act_p), label=["new act x", "new act y", "new act z"], marker='o', markersize=0.2)
            ax[1,0].legend(loc="upper right", fontsize=18)
            ax[1,0].set_xlabel("Actual pose in F frame")
            ax[1,0].grid()
            ax[1,0].set_xlim(0)
            ax[1,0].minorticks_on()

            ax[1,1].plot(self.solt, np.array(plot_est_p), label=["new est x", "new est y", "new est z"], marker='o', markersize=0.2)
            ax[1,1].legend(loc="upper right", fontsize=18)
            ax[1,1].set_xlabel("Estimated pose in F frame")
            ax[1,1].grid()
            ax[1,1].set_xlim(0)
            ax[1,1].minorticks_on()

            # ax[2,0].scatter(self.solt[:-1], self.solnumlandmark, marker='o', s=1)
            # ax[2,0].set_xlabel("number of landmark")
            # ax[2,0].grid()
            # ax[2,0].set_xlim(0)
            # ax[2,0].minorticks_on()

            # dttemp = []
            # for idxtemp, ttemp in enumerate(self.solt):
            #     if idxtemp < len(self.solt)-1:
            #         dttemp.append(self.solt[idxtemp+1] - self.solt[idxtemp])
            # ax[3,1].plot(self.solt[:-1], dttemp, color="blue", alpha=0.8, label="dt between each timestep")
            # ax[3,1].plot(self.solt, self.caltime, color="orange", alpha=0.8, label="cal time")
            # ax[3,1].legend(loc="upper right")
            # ax[3,1].set_xlabel("time")
            # ax[3,1].grid()
            # ax[3,1].set_xlim(0)
            # ax[3,1].minorticks_on()
            param = {'agent name' : self.name,
                     'use_adaptive': self.use_adaptive,
                     'quaternion': self.quaternion,
                     'time': self.time,
                     'stepsize': self.stepsize,
                     'tol': self.tol,
                     'noise': self.noise,
                     'which_eq': self.which_eq,
                     'which_omega': self.which_omega,
                     'with_image_hz_sim': self.with_image_hz_sim,
                     'image_hz': self.image_hz,
                     'randomize_image_input': self.randomize_image_input,
                     'number of landmark': self.l,
                     'k': self.k,
                     'q': self.q[0],
                     'v': self.v,
                     'P': self.p_ricatti}

            param_text = "\n".join([f"{k} = {v}" for k, v in param.items()])
            ax[2,0].xaxis.set_visible(False)
            ax[2,0].yaxis.set_visible(False)
            ax[2,0].set_frame_on(False)
            ax[2,0].text(0.5, 0.5, param_text, fontsize=10, va='center', ha='left',
              bbox=dict(boxstyle="round", alpha=0.1))
            plt.tight_layout(pad=2.0)
            figure.suptitle(f"{self.name}")
        except Exception as e:
            print(e)
            

        ax3d = plt.figure(figsize=(15,15)).add_subplot(projection='3d')
        ax3d.plot(np.array(plot_est_p)[:,0], np.array(plot_est_p)[:,1], np.array(plot_est_p)[:,2], label='Estimated pose for ' + self.name)
        ax3d.plot(np.array(plot_est_p)[0,0], np.array(plot_est_p)[0,1], np.array(plot_est_p)[0,2], ms=5, marker="o") # , label='Estimtaed poes for ' + self.agent_name
        ax3d.plot(np.array(plot_act_p)[:,0], np.array(plot_act_p)[:,1], np.array(plot_act_p)[:,2], label='Actual pose for ' + self.name, linestyle="dotted")
        ax3d.plot(np.array(plot_act_p)[0,0], np.array(plot_act_p)[0,1], np.array(plot_act_p)[0,2], ms=5, marker="o") # , label='Actual pose start for ' + self.agent_name
        ax3d.legend()

        return figure, ax, plot_act_lambda_bar, plot_act_p_bar
        ###############################################################################################################################
        ###############################################################################################################################

    def observer_equations(self, input_omega, input_p_bar_hat, input_R, input_v, num_landmarks, input_q, input_R_hat, input_z, input_p, with_noise, input_k, input_P, which_eq, input_z_act):
        # omega
        first_upper = input_omega
        
        # -S(omega)p_bat_hat + v_bar
        first_lower = -np.matmul(self.function_S(input_omega), input_p_bar_hat) + self.add_bar(input_R, input_v)
        first_part = np.vstack((first_upper, first_lower))
        # omega_hat second part upper
        final = np.transpose(np.array([[0, 0, 0]], dtype=np.float64))
        final2 = np.transpose(np.array([[0, 0, 0]], dtype=np.float64))
        for landmark_idx in range(num_landmarks):
            #q*S(R_hat.T z)
            first = input_q*self.function_S(np.matmul(np.transpose(input_R_hat), np.transpose(input_z[landmark_idx])))
            #Pi_d
            Pi_d = self.function_Pi(self.function_d(input_R, input_p, np.transpose(input_z_act[landmark_idx]), with_noise)) #TODO
            #(p_bar_hat - R_hat.T x z)
            second = input_p_bar_hat - np.matmul(np.transpose(input_R_hat), np.transpose(input_z[landmark_idx]))
            final += np.matmul(first, np.matmul(Pi_d, second))
            # omega_hat second part lower
            #q*Pi_d
            first = input_q*Pi_d
            #(p_bar_hat - R_hat.T x z)
            # second = input_p_bar_hat - np.matmul(np.transpose(input_R_hat), np.transpose(input_z[landmark_idx]))
            final2 += np.matmul(first, second)
        second_part = np.vstack((final, final2))
        #kP[]
        #full second part 
        second_part = input_k*np.matmul(input_P, second_part)

        # Final
        output_omega_hat_p_bar_hat_dot = first_part - second_part
            
    
        return output_omega_hat_p_bar_hat_dot

    def dynamics(self, t, y, input_k, input_q, input_Q, input_V, with_noise, num_landmarks, which_eq, quaternion, which_omega):
        # pose
        input_p = self.GetPose(t)
        ####################################
        ########### Measurements ###########
        if with_noise:
            print(f"NO With Noise function available")
            pass
        else:
            # velocity
            input_v = self.GetlinVelocity(t)
            # angular velocity
            input_omega = self.GetAngVelocity(t)
        ########### Measurements ###########
        ####################################
        if quaternion:
            ####################################
            ############ Quaternion ############
            qua_flat, qua_hat_flat, p_bar_hat_flat, input_P_flat = np.split(y, [4, 8, 11])
            qua_flat = qua_flat/np.linalg.norm(qua_flat)
            qua_hat_flat = qua_hat_flat/np.linalg.norm(qua_hat_flat)
            input_R = self.rodrigues_formula(qua_flat)
            input_R_hat = self.rodrigues_formula(qua_hat_flat)
        ############ Quaternion ############
        ####################################
        else:
            ####################################
            ######### Rotation Matrix ##########
            Rot_flat, Rot_hat_flat, p_bar_hat_flat, input_P_flat = np.split(y, [9, 18, 21])
            input_R = Rot_flat.reshape((3,3))
            input_R_hat = Rot_hat_flat.reshape((3,3))
            ######### Rotation Matrix ##########
            ####################################

        input_z = self.GetLandmark(t)
        input_z_act = self.GetLandmark_act(t)
        
        input_p_bar_hat = p_bar_hat_flat.reshape((3,1))
        input_P = input_P_flat.reshape((6,6))

        input_A = self.function_A(input_omega)
        input_C = self.function_C(input_R, input_R_hat, input_p, input_z, input_z_act, with_noise, num_landmarks)
        ####################################

        ####################################
        ############# Observer #############
        output_omega_hat_p_bar_hat_dot = self.observer_equations(input_omega, input_p_bar_hat, input_R, input_v, num_landmarks, input_q, input_R_hat, input_z, input_p, with_noise, input_k, input_P, which_eq, input_z_act)
        # output_omega_hat_p_bar_hat_dot[:2,:] = 0.0
        ############# Observer #############
        ####################################
        output_P_dot = np.matmul(input_A, input_P) + np.matmul(input_P, np.transpose(input_A)) - np.matmul(input_P, np.matmul(np.transpose(input_C), np.matmul(input_Q, np.matmul(input_C, input_P)))) + input_V
        p_bar_hat_dot = output_omega_hat_p_bar_hat_dot[3:]

        if quaternion:
            ####################################
            ############ Quaternion ############
            omega_hat = output_omega_hat_p_bar_hat_dot[0:3].flatten()
            omega_hat_4x4 = np.array([[0, -omega_hat[0], -omega_hat[1], -omega_hat[2]],
                                    [omega_hat[0], 0, omega_hat[2], -omega_hat[1]],
                                    [omega_hat[1], -omega_hat[2], 0, omega_hat[0]],
                                    [omega_hat[2], omega_hat[1], -omega_hat[0], 0]])

            output_qua_hat_flat = 0.5*np.matmul(omega_hat_4x4, qua_hat_flat)
            input_omega = input_omega.flatten()
            omega_4x4 = np.array([[0, -input_omega[0], -input_omega[1], -input_omega[2]],
                                    [input_omega[0], 0, input_omega[2], -input_omega[1]],
                                    [input_omega[1], -input_omega[2], 0, input_omega[0]],
                                    [input_omega[2], input_omega[1], -input_omega[0], 0]])
            output_qua_flat = 0.5*np.matmul(omega_4x4, qua_flat)
            return np.concatenate((output_qua_flat, output_qua_hat_flat, p_bar_hat_dot.flatten(), output_P_dot.flatten()))
            ########### Quaternion ############
            ####################################
        else:
            ####################################
            ######### Rotation Matrix ##########
            omega_hat = output_omega_hat_p_bar_hat_dot[0:3]
            output_R_hat_dot = np.matmul(input_R_hat, self.function_S(omega_hat))
            output_R = np.matmul(input_R, self.function_S(input_omega))

            # (Rot.flatten(), Rot_hat.flatten(), p_bar_hat.flatten(), P_ricatti.flatten())
            return np.concatenate((output_R.flatten(), output_R_hat_dot.flatten(), p_bar_hat_dot.flatten(), output_P_dot.flatten()))
            ######### Rotation Matrix ##########
            ####################################

    def rk45_step(self, t, y, dt, args, tol, adaptive):
        a2, a3, a4, a5, a6 = 1/5, 3/10, 4/5, 8/9, 1
        b21 = 1/5
        b31, b32 = 3/40, 9/40
        b41, b42, b43 = 44/45, -56/15, 32/9
        b51, b52, b53, b54 = 19372/6561, -25360/2187, 64448/6561, -212/729
        b61, b62, b63, b64, b65 = 9017/3168, -355/33, 46732/5247, 49/176, -5103/18656
        c1, c2, c3, c4, c5, c6 = 35/384, 0, 500/1113, 125/192, -2187/6784, 11/84

        c1_4, c3_4, c4_4, c5_4, c6_4 = 5179/57600, 7571/16695, 393/640, -92097/339200, 187/2100
        # Runge-Kutta stages
        k1 = self.dynamics(t, y, *args)
        k2 = self.dynamics(t + a2*dt, y + dt*b21*k1, *args)
        k3 = self.dynamics(t + a3*dt, y + dt*(b31*k1 + b32*k2), *args)
        k4 = self.dynamics(t + a4*dt, y + dt*(b41*k1 + b42*k2 + b43*k3), *args)
        k5 = self.dynamics(t + a5*dt, y + dt*(b51*k1 + b52*k2 + b53*k3 + b54*k4), *args)
        k6 = self.dynamics(t + a6*dt, y + dt*(b61*k1 + b62*k2 + b63*k3 + b64*k4 + b65*k5), *args)

        # Update step
        y_next = y + dt*(c1*k1 + c2*k2 + c3*k3 + c4*k4 + c5*k5 + c6*k6)

        if adaptive:
            y_next_4 = y + dt * (c1_4*k1 + c3_4*k3 + c4_4*k4 + c5_4*k5 + c6_4*k6)

            error = np.abs(y_next - y_next_4)
            error_norm = np.linalg.norm(error)
            safety_factor = 0.9
            min_scale_factor = 0.2
            max_scale_factor = 40.0
            if error_norm <= tol:
                success = True
                t = t+dt
                dt = dt * min(max_scale_factor, max(min_scale_factor, safety_factor * (tol / error_norm)**0.25))
            else:
                success = False
                dt = dt * max(min_scale_factor, safety_factor * (tol / error_norm)**0.25)
        else: 
            if np.isnan(y_next).any():
                success = False
            else:
                success = True
        return y_next, t, dt, success

    def step_simulation(self):
        ### Run solver
        ######################################################
        ####################### Solver #######################
        success = False
        while not success:
            if self.with_image_hz_sim:
                # show_measurement = any(abs(current_time - t)/(t+1e-10) <= threshold_image for t in image_time)
                self.show_measurement = np.round(self.current_time, decimals=2) == np.round(self.image_time[min(self.i, len(self.image_time)-1)], decimals=2)
                if self.show_measurement:
                    if self.randomize_image_input:
                        l = np.random.rand(len(self.z_appear),1)
                        l = l/np.linalg.norm(l)
                        l = np.rint(l).astype(int)
                        z = self.z_appear[l[:, 0] == 1]
                        self.l = np.sum(l)
                        self.q = self.q
                        self.Q = np.diag(np.hstack([np.diag(self.q*np.eye(3)) for aa in range(l)]))
                    else:
                        self.l = len(self.z_appear)
                        z = self.z_appear
                else:
                    self.l = 0
                    z = np.array([None])
            else:
                z = self.z_appear

            args = (self.k, z, self.q, self.Q, self.V, self.noise, self.l, self.which_eq, self.quaternion, self.which_omega)
            y_next, next_time, new_dt, success = self.rk45_step(self.current_time, self.soly[-1], self.dt, args, self.tol, self.use_adaptive)
            if self.use_adaptive:
                if success:
                    self.soly.append(y_next)
                    self.solt.append(self.current_time)
                    self.solimage.append([self.current_time, self.show_measurement])
                    self.solnumlandmark.append(self.l)
                    end_time = systemtime.time()
                    # self.caltime.append(end_time - start_time)
                    # start_time = end_time
                    if self.with_image_hz_sim:
                        if self.show_measurement:
                            self.i += 1
                        if next_time + new_dt < self.image_time[min(self.i, len(self.image_time)-1)]:
                            self.dt = new_dt if new_dt != 0 else min(self.stepsize, 1/self.image_hz)
                        else:
                            self.dt = abs(self.image_time[min(self.i, len(self.image_time)-1)] - next_time)
                            self.current_time = self.image_time[self.i]
                    else:
                        self.dt = new_dt
                        self.current_time = next_time
                else:
                    self.dt = new_dt
            else:
                if success:
                    self.soly.append(y_next)
                    self.solt.append(self.current_time)
                    end_time = systemtime.time()
                    # self.caltime.append(end_time - start_time)
                    # start_time = end_time
                    self.current_time += self.dt

            ####################### Solver #######################
            ######################################################
        return (self.solt[-1], self.dt, self.soly[-1])

    def full_simulation(self):
        ### Run solver
        ######################################################
        ####################### Solver #######################
        start_time = systemtime.time()
        time_zero = start_time
        run_time = systemtime.time()
        while self.current_time <= self.time[1]:
            # print(f"Simulation time: {current_time} Run time: {systemtime.time()-run_time}", end='\r', flush=True) 
            simulation_time_str = f"Simulation time: {self.current_time}"
            print(f"{simulation_time_str}{' ' * (40 - len(simulation_time_str))}Run time: {systemtime.time()-run_time}", end='\r', flush=True)
           
            args = (self.k, self.q, self.Q, self.V, self.noise, self.l, self.which_eq, self.quaternion, self.which_omega)
            y_next, next_time, new_dt, success = self.rk45_step(self.current_time, self.soly[-1], self.dt, args, self.tol, self.use_adaptive)
            if self.use_adaptive:
                if success:
                    self.soly.append(y_next)
                    self.solt.append(self.current_time)
                    self.solimage.append([self.current_time, self.show_measurement])
                    self.solnumlandmark.append(self.l)
                    end_time = systemtime.time()
                    self.caltime.append(end_time - start_time)
                    start_time = end_time
                    if next_time >= self.time[1]:
                        self.solt.append(self.time[1])
                        break
                    self.dt = new_dt
                    self.current_time = next_time
                else:
                    self.dt = new_dt
            else:
                if success:
                    self.soly.append(y_next)
                    self.solt.append(self.current_time)
                    end_time = systemtime.time()
                    self.caltime.append(end_time - start_time)
                    start_time = end_time
                    self.current_time += self.dt
                else:
                    print(self.current_time, len(self.soly))
                    break
        ####################### Solver #######################
        ######################################################
        return self.solt, self.soly, self.caltime, args

#### CASES

In [ ]:
def original_setting(save_fig=False, need_y_lim=False, time=(0,1)):
    landmark = lambda t: ([[[5,0,0]],[[0,0,0]], [[2.5,2.5,0*t]]])
    l = 3
    p_formula = lambda t: np.transpose(np.array([[2.5 + 2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10]]))
    omega_formula = lambda t: np.transpose(np.array([[0.1*np.sin(t), 0.4*np.cos(2*t), 0.6*t]]))
    v_formula = lambda t: np.transpose(np.array([[-np.sin(0.4*t), np.cos(0.4*t), 0]]))

    use_adaptive            = True
    quaternion              = True
    stepsize                = 0.001
    tol                     = 1e-3
    noise                   = False
    which_eq                = 0
    which_omega             = "full" # "z" or "full"
    with_image_hz_sim       = False
    image_hz                = 60
    randomize_image_input   = False
    p_hat                   = np.array([[-2, 4, 3]], dtype=np.float64).T
    qua = [np.sqrt(2)/2, np.sqrt(2)/2, 0, 0]
    qua /= np.linalg.norm(qua)
    Lambda_bar_0            = np.array(qua, dtype=np.float64).T  # quaternion: w, x, y, z
    z_appear                = landmark
    z_appear_act            = landmark
    name = "agent1"
    # np.array([[[0, 0, 0]], 
                                        # [[5, 0, 0]], 
                                        # [[2.5, 2.5, 0]]])
    p_ricatti               = np.array([1, 100])


    agent1 = riccati_observer(
        use_adaptive            = use_adaptive,
        quaternion              = quaternion,
        p_formula               = p_formula,
        omega_formula           = omega_formula,
        v_formula               = v_formula,
        time                    = time,
        stepsize                = stepsize,
        tol                     = tol,
        noise                   = noise,
        which_eq                = which_eq,
        which_omega             = which_omega,
        with_image_hz_sim       = with_image_hz_sim, 
        image_hz                = image_hz, 
        randomize_image_input   = randomize_image_input, 
        p_hat                   = p_hat,
        Lambda_bar_0            = Lambda_bar_0,  # quaternion: w, x, y, z
        z_appear                = z_appear,
        z_appear_act            = z_appear_act,
        l                       = l,
        k                       = 1,
        q                       = [10], 
        v                       = np.array([0.1, 1]),
        p_ricatti               = p_ricatti,
        name                    = name
        )
    
    agent1.full_simulation()
    print("Plotting")
    fig, ax, lambda_, p_bar = agent1.visual_plot(figsize=(12,12), bound_y=True)  
    return agent1

In [ ]:
# agentOriginal = original_setting(time=(0,0.0001))

In [ ]:
def cascade_setting(first_agent=None, second_agent=None, save_fig=False, need_y_lim=False, time=(0,1)):
    print("agent1")
    if first_agent == None:
        l = 3
        p_formula = lambda t: np.transpose(np.array([[2 + 3*np.sin(t), 1+np.sin(3*t), 10]]))
        v_formula = lambda t: np.transpose(np.array([[3*np.cos(t), 3*np.cos(3*t), 0]]))
        omega_formula = lambda t: np.transpose(np.array([[0.1*np.cos(0.1*t), 0, 0]]))

        use_adaptive            = True
        quaternion              = True
        stepsize                = 0.0001
        tol                     = 1e-3
        noise                   = False
        which_eq                = 0
        which_omega             = "full" # "z" or "full"
        with_image_hz_sim       = False
        image_hz                = 60
        randomize_image_input   = False
        p_hat                   = np.array([[-5, 5, 2]], dtype=np.float64).T
        qua = [0.8660254, 0, 0, 0.5]
        qua /= np.linalg.norm(qua)
        Lambda_bar_0            = np.array(qua, dtype=np.float64).T  # quaternion: w, x, y, z
        z_appear                = lambda t: ([[[0,0,0]],[[5,-2,0]],[[3, 3, 0]]])
        z_appear_act            = z_appear
        k                       = 1
        q                       = [10]
        p_ricatti               = np.array([1, 100])
        first_name = "Follower 1"


        agent_first_follower = riccati_observer(
            use_adaptive            = use_adaptive,
            quaternion              = quaternion,
            p_formula               = p_formula,
            omega_formula           = omega_formula,
            v_formula               = v_formula,
            time                    = time,
            stepsize                = stepsize,
            tol                     = tol,
            noise                   = noise,
            which_eq                = which_eq,
            which_omega             = which_omega,
            with_image_hz_sim       = with_image_hz_sim, 
            image_hz                = image_hz, 
            randomize_image_input   = randomize_image_input, 
            p_hat                   = p_hat,
            Lambda_bar_0            = Lambda_bar_0,  # quaternion: w, x, y, z
            z_appear                = z_appear,
            z_appear_act            = z_appear_act,
            l                       = l,
            k                       = k,
            q                       = q, 
            v                       = np.array([0.1, 1]),
            p_ricatti               = p_ricatti,
            name                    = first_name

            )
        
        first_solt, first_soly, first_caltime, first_args = agent_first_follower.full_simulation()
    else:
        first_solt = first_agent.solt
        first_soly = first_agent.soly

    ############### INTERPOLATION ###############
    first_est_p = []
    agent_first_solution = np.array(first_soly)[:, 4:11]
    for idx, (t, solution) in enumerate(zip(first_solt, first_soly)):
        if first_agent == None:
            own_sol_est_R = agent_first_follower.rodrigues_formula(solution[4:8])
        else:
            own_sol_est_R = first_agent.rodrigues_formula(solution[4:8])
        own_sol_est_p_bar = solution[8:11]
        p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
        first_est_p.append(p_temp)
    first_est_p = np.array(first_est_p)
    first_agent_interpolation = CubicSpline(first_solt, first_est_p)
    ############### SECOND FOLLOWER ###############


    print("agent2")
    if second_agent == None:
        second_p_formula = lambda t: np.transpose(np.array([[6+np.cos(3*t), 1+3*np.cos(t), 20]]))
        second_v_formula = lambda t: np.transpose(np.array([[-3*np.sin(3*t), -3*np.sin(t), 0]]))
        second_omega_formula = lambda t: np.transpose(np.array([[0, 0.1*np.cos(0.1*t), 0]]))

        second_use_adaptive            = True
        second_quaternion              = True
        second_stepsize                = 0.001
        second_tol                     = 1e-3
        second_noise                   = False
        second_which_eq                = 0
        second_which_omega             = "full" # "z" or "full"
        second_with_image_hz_sim       = False
        second_image_hz                = 60
        second_randomize_image_input   = False
        second_p_hat                   = np.array([[-2, 5, 10]], dtype=np.float64).T
        second_qua = [0.8660254, 0, 0, 0.5]
        second_qua /= np.linalg.norm(second_qua)
        second_Lambda_bar_0            = np.array(second_qua, dtype=np.float64).T  # quaternion: w, x, y, z
        second_z_appear                = lambda t: ([[[5, -2, 0]], [[3,3,0]],[first_agent_interpolation(t)]])
        second_z_appear_act            = lambda t: ([[[5, -2, 0]], [[3,3,0]],[[2 + 3*np.sin(t), 1+np.sin(3*t), 10]]])
        second_l                       = 3
        second_k                       = 1
        second_q                       = [10]
        second_p_ricatti               = np.array([1, 100])
        second_name = "Follower 2"

        agent_second_follower = riccati_observer(
            use_adaptive            = second_use_adaptive,
            quaternion              = second_quaternion,
            p_formula               = second_p_formula,
            omega_formula           = second_omega_formula,
            v_formula               = second_v_formula,
            time                    = time,
            stepsize                = second_stepsize,
            tol                     = second_tol,
            noise                   = second_noise,
            which_eq                = second_which_eq,
            which_omega             = second_which_omega,
            with_image_hz_sim       = second_with_image_hz_sim, 
            image_hz                = second_image_hz, 
            randomize_image_input   = second_randomize_image_input, 
            p_hat                   = second_p_hat,
            Lambda_bar_0            = second_Lambda_bar_0,  # quaternion: w, x, y, z
            z_appear                = second_z_appear,
            z_appear_act            = second_z_appear_act,
            l                       = second_l,
            k                       = second_k,
            q                       = second_q, 
            v                       = np.array([0.1, 1]),
            p_ricatti               = second_p_ricatti,
            name                    = second_name
            )
        
        # second_solt, second_soly, second_caltime, second_args = agent_second_follower.full_simulation()
    else:
        second_solt = second_agent.solt
        second_soly = second_agent.soly

    ############### INTERPOLATION ###############
    second_est_p = []
    # agent_second_solution = np.array(second_soly)[:, 4:11]
    # for idx, (t, solution) in enumerate(zip(second_solt, second_soly)):
        # own_sol_est_R = second_agent.rodrigues_formula(solution[4:8])
        # own_sol_est_p_bar = solution[8:11]
        # p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
        # second_est_p.append(p_temp)
    # second_est_p = np.array(second_est_p)
    # second_agent_interpolation = CubicSpline(second_solt, second_est_p)
    ############### THIRD FOLLOWER ###############


    print("agent3")
    third_p_formula = lambda t: np.transpose(np.array([[2+3*np.cos(t),-5+np.cos(3*t), -15]]))
    third_v_formula = lambda t: np.transpose(np.array([[-3*np.sin(t), -3*np.sin(3*t), 0]]))
    third_omega_formula = lambda t: np.transpose(np.array([[0, 0, 0.1*np.cos(0.1*t)]]))

    third_use_adaptive            = True
    third_quaternion              = True
    third_stepsize                = 0.001
    third_tol                     = 1e-3
    third_noise                   = False
    third_which_eq                = 0
    third_which_omega             = "full" # "z" or "full"
    third_with_image_hz_sim       = False
    third_image_hz                = 60
    third_randomize_image_input   = False
    third_p_hat                   = np.array([[-2, 4, 5]], dtype=np.float64).T
    third_qua = [0.8660254, 0, 0, 0.5]
    third_qua /= np.linalg.norm(third_qua)
    third_Lambda_bar_0            = np.array(third_qua, dtype=np.float64).T  # quaternion: w, x, y, z
    third_z_appear                = lambda t: ([[[5, -2, 0]], [[3,3,0]],[second_agent_interpolation(t)]])
    third_z_appear_act            = lambda t: ([[[5, -2, 0]], [[3,3,0]],[[6+np.cos(3*t), 1+3*np.cos(t), 20]]])
    third_l                       = 3
    third_k                       = 1
    third_q                       = [10]
    third_p_ricatti               = np.array([1, 100])
    third_name = "Follower 3"


    agent_third_follower = riccati_observer(
        use_adaptive            = third_use_adaptive,
        quaternion              = third_quaternion,
        p_formula               = third_p_formula,
        omega_formula           = third_omega_formula,
        v_formula               = third_v_formula,
        time                    = time,
        stepsize                = third_stepsize,
        tol                     = third_tol,
        noise                   = third_noise,
        which_eq                = third_which_eq,
        which_omega             = third_which_omega,
        with_image_hz_sim       = third_with_image_hz_sim, 
        image_hz                = third_image_hz, 
        randomize_image_input   = third_randomize_image_input, 
        p_hat                   = third_p_hat,
        Lambda_bar_0            = third_Lambda_bar_0,  # quaternion: w, x, y, z
        z_appear                = third_z_appear,
        z_appear_act            = third_z_appear_act,
        l                       = third_l,
        k                       = third_k,
        q                       = third_q, 
        v                       = np.array([0.1, 1]),
        p_ricatti               = third_p_ricatti,
        name                    = third_name
        )
        
    # third_solt, third_soly, third_caltime, third_args = agent_third_follower.full_simulation()


    print("Plotting")
    if first_agent == None:
        fig, ax, lambda_, p_bar = agent_first_follower.visual_plot(figsize=(12,12), bound_y=True)  
    else:
        fig, ax, lambda_, p_bar = first_agent.visual_plot(figsize=(12,12), bound_y=True)  

    # if second_agent == None:
        # fig, ax, lambda_, p_bar = second_agent.visual_plot(figsize=(12,12), bound_y=True)  
    
    # fig, ax, lambda_, p_bar = agent_third_follower.visual_plot(figsize=(12,12), bound_y=True)  
    # return (first_agent, second_agent, agent_third_follower)
    return (agent_first_follower, agent_second_follower, agent_third_follower)


In [ ]:
# cascade_agents = cascade_setting(time=(0,30)) #first_agent=cascade_agents[0], second_agent=cascade_agents[1], 

In [ ]:
def cascade_straight_line_setting(first_agent=None, second_agent=None, third_agent=None, save_fig=False, need_y_lim=False, time=(0,1)):
    print("agent1")
    if first_agent == None:

        p_formula = lambda t: np.transpose(np.array([[-5 + 0.6*(t), -15+0.6*(t), 2]]))
        v_formula = lambda t: np.transpose(np.array([[0.6, 0.6, 0]]))
        omega_formula = lambda t: np.transpose(np.array([[np.cos(t), np.cos(t), np.cos(t)]]))

        use_adaptive            = True
        quaternion              = True
        stepsize                = 0.0001
        tol                     = 1e-3
        noise                   = False
        which_eq                = 0
        which_omega             = "full" # "z" or "full"
        with_image_hz_sim       = False
        image_hz                = 60
        randomize_image_input   = False
        p_hat                   = np.array([[0, -5, 5]], dtype=np.float64).T
        qua = [0.7071068, 0, 0, 0.7071068]
        qua /= np.linalg.norm(qua)
        Lambda_bar_0            = np.array(qua, dtype=np.float64).T  # quaternion: w, x, y, z
        z_appear                = lambda t: ([[[0,0,0]],[[5,-2,0]],[[3, 3, 3]]])
        z_appear_act            = z_appear
        l                       = 3
        k                       = 1
        q                       = [10]
        p_ricatti               = np.array([1, 100])
        first_name = "Follower 1"


        agent_first_follower = riccati_observer(
            use_adaptive            = use_adaptive,
            quaternion              = quaternion,
            p_formula               = p_formula,
            omega_formula           = omega_formula,
            v_formula               = v_formula,
            time                    = time,
            stepsize                = stepsize,
            tol                     = tol,
            noise                   = noise,
            which_eq                = which_eq,
            which_omega             = which_omega,
            with_image_hz_sim       = with_image_hz_sim, 
            image_hz                = image_hz, 
            randomize_image_input   = randomize_image_input, 
            p_hat                   = p_hat,
            Lambda_bar_0            = Lambda_bar_0,  # quaternion: w, x, y, z
            z_appear                = z_appear,
            z_appear_act            = z_appear_act,
            l                       = l,
            k                       = k,
            q                       = q, 
            v                       = np.array([0.1, 1]),
            p_ricatti               = p_ricatti,
            name                    = first_name

            )
        
        first_solt, first_soly, first_caltime, first_args = agent_first_follower.full_simulation()

        ############### INTERPOLATION ###############
        first_est_p = []
        agent_first_solution = np.array(first_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(first_solt, first_soly)):
            if first_agent == None:
                own_sol_est_R = agent_first_follower.rodrigues_formula(solution[4:8])
            else:
                own_sol_est_R = first_agent.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            first_est_p.append(p_temp)
        first_est_p = np.array(first_est_p)
        first_agent_interpolation = CubicSpline(first_solt, first_est_p)
        ############### SECOND FOLLOWER ###############

    else:
        first_solt = first_agent.solt
        first_soly = first_agent.soly
        ############### INTERPOLATION ###############
        first_est_p = []
        agent_first_solution = np.array(first_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(first_solt, first_soly)):
            if first_agent == None:
                own_sol_est_R = first_agent.rodrigues_formula(solution[4:8])
            else:
                own_sol_est_R = first_agent.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            first_est_p.append(p_temp)
        first_est_p = np.array(first_est_p)
        first_agent_interpolation = CubicSpline(first_solt, first_est_p)
        ############### SECOND FOLLOWER ###############


    print("agent2")
    if second_agent == None:
        second_p_formula = lambda t: np.transpose(np.array([[-6+0.6*t, 3-0.2*t, 3]]))
        second_v_formula = lambda t: np.transpose(np.array([[0.6, -0.2, 0]]))
        second_omega_formula = lambda t: np.transpose(np.array([[np.cos(t), np.cos(t), np.cos(t)]]))

        second_use_adaptive            = True
        second_quaternion              = True
        second_stepsize                = 0.001
        second_tol                     = 1e-3
        second_noise                   = False
        second_which_eq                = 0
        second_which_omega             = "full" # "z" or "full"
        second_with_image_hz_sim       = False
        second_image_hz                = 60
        second_randomize_image_input   = False
        second_p_hat                   = np.array([[-1, 13, 6]], dtype=np.float64).T
        second_qua = [0.7071068, 0, 0, 0.7071068]
        second_qua /= np.linalg.norm(second_qua)
        second_Lambda_bar_0            = np.array(second_qua, dtype=np.float64).T  # quaternion: w, x, y, z
        second_z_appear                = lambda t: ([[[5, -2, 0]], [[0, 0, 0]], [first_agent_interpolation(t)]])
        second_z_appear_act            = lambda t: ([[[5, -2, 0]], [[0, 0, 0]], [[-5 + 0.6*(t), -15+0.6*(t), 2]]])
        second_l                       = 3
        second_k                       = 1
        second_q                       = [10]
        second_p_ricatti               = np.array([1, 100])
        second_name = "Follower 2"

        agent_second_follower = riccati_observer(
            use_adaptive            = second_use_adaptive,
            quaternion              = second_quaternion,
            p_formula               = second_p_formula,
            omega_formula           = second_omega_formula,
            v_formula               = second_v_formula,
            time                    = time,
            stepsize                = second_stepsize,
            tol                     = second_tol,
            noise                   = second_noise,
            which_eq                = second_which_eq,
            which_omega             = second_which_omega,
            with_image_hz_sim       = second_with_image_hz_sim, 
            image_hz                = second_image_hz, 
            randomize_image_input   = second_randomize_image_input, 
            p_hat                   = second_p_hat,
            Lambda_bar_0            = second_Lambda_bar_0,  # quaternion: w, x, y, z
            z_appear                = second_z_appear,
            z_appear_act            = second_z_appear_act,
            l                       = second_l,
            k                       = second_k,
            q                       = second_q, 
            v                       = np.array([0.1, 1]),
            p_ricatti               = second_p_ricatti,
            name                    = second_name
            )
        
        second_solt, second_soly, second_caltime, second_args = agent_second_follower.full_simulation()
        ############### INTERPOLATION ###############
        # second_est_p = []
        # agent_second_solution = np.array(second_soly)[:, 4:11]
        # for idx, (t, solution) in enumerate(zip(second_solt, second_soly)):
        #     own_sol_est_R = agent_second_follower.rodrigues_formula(solution[4:8])
        #     own_sol_est_p_bar = solution[8:11]
        #     p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
        #     second_est_p.append(p_temp)
        # second_est_p = np.array(second_est_p)
        # second_agent_interpolation = CubicSpline(second_solt, second_est_p)
        ############### THIRD FOLLOWER ###############
    else:
        second_solt = second_agent.solt
        second_soly = second_agent.soly
        ############### INTERPOLATION ###############
        second_est_p = []
        agent_second_solution = np.array(second_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(second_solt, second_soly)):
            own_sol_est_R = second_agent.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            second_est_p.append(p_temp)
        second_est_p = np.array(second_est_p)
        second_agent_interpolation = CubicSpline(second_solt, second_est_p)
        ############### THIRD FOLLOWER ###############


    print("agent3")
    if third_agent == None:
        third_p_formula = lambda t: np.transpose(np.array([[4,10-0.6*t, 2]]))
        third_v_formula = lambda t: np.transpose(np.array([[0, -0.6, 0]]))
        third_omega_formula = lambda t: np.transpose(np.array([[np.cos(t), np.cos(t), np.cos(t)]]))

        third_use_adaptive            = True
        third_quaternion              = True
        third_stepsize                = 0.001
        third_tol                     = 1e-3
        third_noise                   = False
        third_which_eq                = 0
        third_which_omega             = "full" # "z" or "full"
        third_with_image_hz_sim       = False
        third_image_hz                = 60
        third_randomize_image_input   = False
        third_p_hat                   = np.array([[-6, -4, 5]], dtype=np.float64).T
        third_qua = [0.7071068, 0, 0, 0.7071068]
        third_qua /= np.linalg.norm(third_qua)
        third_Lambda_bar_0            = np.array(third_qua, dtype=np.float64).T  # quaternion: w, x, y, z
        third_z_appear                = lambda t: ([[[5, -2, 0]], [[0, 0,0]],[second_agent_interpolation(t)]])
        third_z_appear_act            = lambda t: ([[[5, -2, 0]], [[0, 0,0]],[[-6+0.6*t, 3-0.1*t, 3]]])
        third_l                       = 3
        third_k                       = 1
        third_q                       = [10]
        third_p_ricatti               = np.array([1, 100])
        third_name = "Follower 3"


        agent_third_follower = riccati_observer(
            use_adaptive            = third_use_adaptive,
            quaternion              = third_quaternion,
            p_formula               = third_p_formula,
            omega_formula           = third_omega_formula,
            v_formula               = third_v_formula,
            time                    = time,
            stepsize                = third_stepsize,
            tol                     = third_tol,
            noise                   = third_noise,
            which_eq                = third_which_eq,
            which_omega             = third_which_omega,
            with_image_hz_sim       = third_with_image_hz_sim, 
            image_hz                = third_image_hz, 
            randomize_image_input   = third_randomize_image_input, 
            p_hat                   = third_p_hat,
            Lambda_bar_0            = third_Lambda_bar_0,  # quaternion: w, x, y, z
            z_appear                = third_z_appear,
            z_appear_act            = third_z_appear_act,
            l                       = third_l,
            k                       = third_k,
            q                       = third_q, 
            v                       = np.array([0.1, 1]),
            p_ricatti               = third_p_ricatti,
            name                    = third_name
            )
            
        # third_solt, third_soly, third_caltime, third_args = agent_third_follower.full_simulation()


    print("Plotting")
    if first_agent == None:
        fig, ax, lambda_, p_bar = agent_first_follower.visual_plot(figsize=(12,12), bound_y=True) 
    else:
        fig, ax, lambda_, p_bar = first_agent.visual_plot(figsize=(12,12), bound_y=True)  
        agent_first_follower = first_agent 

    if second_agent == None:
        ig, ax, lambda_, p_bar = agent_second_follower.visual_plot(figsize=(12,12), bound_y=True)  
    else:
        fig, ax, lambda_, p_bar = second_agent.visual_plot(figsize=(12,12), bound_y=True)  
        # agent_second_follower = second_agent
    # if third_agent == None:
        # ig, ax, lambda_, p_bar = agent_third_follower.visual_plot(figsize=(12,12), bound_y=True)  
    # else:
        # fig, ax, lambda_, p_bar = third_agent.visual_plot(figsize=(12,12), bound_y=True)  
        # agent_third_follower = third_agent

    return (first_agent, agent_second_follower, agent_third_follower)


In [ ]:
# cascade_straight_line_agents = cascade_straight_line_setting(first_agent=cascade_straight_line_agents[0],time=(0, 15)) #first_agent=cascade_straight_line_agents[0],second_agent=cascade_straight_line_agents[1], third_agent=cascade_straight_line_agents[2] ,

In [ ]:
# cascade_straight_line_agents[2].visual_plot(figsize=(12,12), bound_y=True)

In [ ]:
# agent_1 = cascade_intersection_agents[0]
# agent_2 = cascade_intersection_agents[1]
# agent_3 = cascade_intersection_agents[2]

In [ ]:
# agent_1_plot_est_p = []
# agent_1_plot_act_p = []
# agent_1_plt_err_degree = []
# for idx, (t, solution) in enumerate(zip(agent_1.solt, agent_1.soly)):
#     own_sol_R = agent_1.rodrigues_formula(solution[:4])
#     own_sol_est_R = agent_1.rodrigues_formula(solution[4:8])
#     own_sol_est_p_bar = solution[8:11]
#     p = agent_1.GetPose(t).reshape((3,))
#     # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
#     p_bar_temp = np.matmul(np.transpose(np.array(own_sol_R).reshape((3,3))), p)
#     p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)

#     agent_1_plot_est_p.append(p_temp.tolist())
#     agent_1_plot_act_p.append(p.tolist())

#     err = np.matmul(np.linalg.inv(np.array(own_sol_est_R).reshape((3,3))), np.array(own_sol_R).reshape((3,3)))
#     err = ScipyRot.from_matrix(err)
#     err_degree = err.as_euler('xyz', degrees=True)
#     agent_1_plt_err_degree.append(err_degree)

In [ ]:
# agent_2_plot_est_p = []
# agent_2_plot_act_p = []
# agent_2_plt_err_degree = []
# for idx, (t, solution) in enumerate(zip(agent_2.solt, agent_2.soly)):
#     own_sol_R = agent_2.rodrigues_formula(solution[:4])
#     own_sol_est_R = agent_2.rodrigues_formula(solution[4:8])
#     own_sol_est_p_bar = solution[8:11]
#     p = agent_2.GetPose(t).reshape((3,))
#     # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
#     p_bar_temp = np.matmul(np.transpose(np.array(own_sol_R).reshape((3,3))), p)
#     p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)

#     agent_2_plot_est_p.append(p_temp.tolist())
#     agent_2_plot_act_p.append(p.tolist())

#     err = np.matmul(np.linalg.inv(np.array(own_sol_est_R).reshape((3,3))), np.array(own_sol_R).reshape((3,3)))
#     err = ScipyRot.from_matrix(err)
#     err_degree = err.as_euler('xyz', degrees=True)
#     agent_2_plt_err_degree.append(err_degree)

In [ ]:
# agent_3_plot_est_p = []
# agent_3_plot_act_p = []
# agent_3_plt_err_degree = []
# for idx, (t, solution) in enumerate(zip(agent_3.solt, agent_3.soly)):
#     own_sol_R = agent_3.rodrigues_formula(solution[:4])
#     own_sol_est_R = agent_3.rodrigues_formula(solution[4:8])
#     own_sol_est_p_bar = solution[8:11]
#     p = agent_3.GetPose(t).reshape((3,))
#     # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
#     p_bar_temp = np.matmul(np.transpose(np.array(own_sol_R).reshape((3,3))), p)
#     p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)

#     agent_3_plot_est_p.append(p_temp.tolist())
#     agent_3_plot_act_p.append(p.tolist())

#     err = np.matmul(np.linalg.inv(np.array(own_sol_est_R).reshape((3,3))), np.array(own_sol_R).reshape((3,3)))
#     err = ScipyRot.from_matrix(err)
#     err_degree = err.as_euler('xyz', degrees=True)
#     agent_3_plt_err_degree.append(err_degree)

In [ ]:
# %matplotlib widget
# current_agent =     agent_2
# plot_act_p =        agent_2_plot_act_p
# plot_est_p =        agent_2_plot_est_p
# plt_err_degree =    agent_2_plt_err_degree

In [ ]:
# own_sol_R = agent_2.rodrigues_formula(agent_2.soly[idx_used][:4])
# own_sol_est_R = agent_2.rodrigues_formula(agent_2.soly[idx_used][4:8])
# own_sol_est_p_bar = agent_2.soly[idx_used][8:11]
# own_sol_est_p_bar

In [ ]:
# timenow = agent_2.solt[idx_used]
# print(np.transpose(agent_2.z_appear_act(timenow)[-1]))
# print(np.transpose(agent_2.z_appear(timenow)[-1]))
# print(np.transpose(agent_2.z_appear_act(timenow)[-1]) - np.transpose(agent_2.z_appear(timenow)[-1]))
# print(agent_1.p_formula(timenow))

In [ ]:
# est_bearing =  (agent_2.p_formula(timenow) - np.transpose(agent_2.z_appear(timenow)[-1]))/np.linalg.norm(agent_2.p_formula(timenow) - np.transpose(agent_2.z_appear(timenow)[-1]))
# act_bearing = (agent_2.p_formula(timenow) - np.transpose(agent_2.z_appear_act(timenow)[-1]))/np.linalg.norm(agent_2.p_formula(timenow) - np.transpose(agent_2.z_appear_act(timenow)[-1]))
# est_bearing =  np.matmul(np.transpose(own_sol_R), est_bearing)
# act_bearing =  np.matmul(np.transpose(own_sol_R), act_bearing)
# est_pi = np.eye(3) - np.matmul(est_bearing, np.transpose(est_bearing))
# act_pi = np.eye(3) - np.matmul(act_bearing, np.transpose(act_bearing))
# print(np.matmul(est_pi, np.matmul(np.transpose(own_sol_est_R), (np.transpose(agent_2.z_appear_act(timenow)[-1]) - np.transpose(agent_2.z_appear(timenow)[-1])))))
# print(np.matmul(act_pi, np.matmul(np.transpose(own_sol_est_R), (np.transpose(agent_2.z_appear_act(timenow)[-1]) - np.transpose(agent_2.z_appear(timenow)[-1])))))

In [ ]:
# np.matmul(np.transpose(own_sol_est_R), np.transpose(agent_2.z_appear(timenow)[-1]))

In [ ]:
# figure, ax = plt.subplots(1,2, figsize=(24,6))
# colorlist = ["tomato", "limegreen", "slateblue"]
# print("hiii")
# if "1" in current_agent.name:
#     name = r"Vehicle $f_1$"
#     ylabel = [r'$\tilde{p}_{f_1}$ (m)' , r'$\tilde{\theta}_{f_1}$ (deg)']
# elif "2" in current_agent.name:
#     name = r"Vehicle $f_2$"
#     ylabel = [r'$\tilde{p}_{f_2}$ (m)' , r'$\tilde{\theta}_{f_2}$ (deg)']
# elif "3" in current_agent.name:
#     name = r"Vehicle $f_3$"
#     ylabel = [r'$\tilde{p}_{f_3}$ (m)' , r'$\tilde{\theta}_{f_3}$ (deg)']

# err_pose = np.array(plot_act_p)-np.array(plot_est_p)
# ax[0].plot(current_agent.solt, err_pose[:,0], label="x error", linewidth=3, color=colorlist[0])
# ax[0].plot(current_agent.solt, err_pose[:,1], label="y error", linewidth=3, color=colorlist[1])
# ax[0].plot(current_agent.solt, err_pose[:,2], label="z error", linewidth=3, color=colorlist[2])
# ax[0].legend(markerscale=15, loc="upper right", fontsize=30) #, bbox_to_anchor=(0.65, 0.85))
# ax[0].set_xlabel("Time (sec)",  fontsize=24)
# ax[0].set_ylabel(ylabel[0],     fontsize=34)
# ax[0].grid()
# ax[0].set_xlim(0, 30)
# ax[0].set_ylim(-0.5, 0.5)
# ax[0].minorticks_on()

# ax[0].tick_params(axis='both', labelsize=28)
# ax[1].tick_params(axis='both', labelsize=28)
# ax[0].yaxis.set_label_coords(-0.12, 0.5)
# ax[1].yaxis.set_label_coords(-0.08, 0)
# # ax[0,1].plot(self.solt, np.array(plot_err_lambda_bar)[:,0:3], label=["x", "y", "z"], marker='o', markersize=0.2, color=colorlist)
# ax[1].plot(current_agent.solt, np.array(plt_err_degree)[:,0], label="roll error", linewidth=3, color=colorlist[0])
# ax[1].plot(current_agent.solt, np.array(plt_err_degree)[:,1], label="pitch error", linewidth=3, color=colorlist[1])
# ax[1].plot(current_agent.solt, np.array(plt_err_degree)[:,2], label="yaw error", linewidth=3, color=colorlist[2])
# ax[1].legend(markerscale=15, loc="upper right", fontsize=28) #, bbox_to_anchor=(0.65, 0.9))
# ax[1].set_xlabel("Time (sec)",  fontsize=24)
# ax[1].set_ylabel(ylabel[1],     fontsize=34)
# ax[1].grid()
# ax[1].set_xlim(0, 30)
# ax[1].set_ylim(-100, 90)
# ax[1].minorticks_on()

# figure.suptitle(f"{name}", fontsize=50)
    


In [ ]:
# %matplotlib widget
# current_agent =     agent_2
# plot_act_p =        agent_2_plot_act_p
# plot_est_p =        agent_2_plot_est_p
# plt_err_degree =    agent_2_plt_err_degree

In [ ]:
# figure, ax = plt.subplots(1,2, figsize=(12,6))
# colorlist = ["tomato", "limegreen", "slateblue"]
# print("hiii")
# if "1" in current_agent.name:
#     name = r"Vehicle $f_1$"
#     ylabel = [r'$\tilde{p}_{f_1}$ (m)' , r'$\tilde{\theta}_{f_1}$ (deg)']
# elif "2" in current_agent.name:
#     name = r"Vehicle $f_2$"
#     ylabel = [r'$\tilde{p}_{f_2}$ (m)' , r'$\tilde{\theta}_{f_2}$ (deg)']
# elif "3" in current_agent.name:
#     name = r"Vehicle $f_3$"
#     ylabel = [r'$\tilde{p}_{f_3}$ (m)' , r'$\tilde{\theta}_{f_3}$ (deg)']

# err_pose = np.array(plot_act_p)-np.array(plot_est_p)
# ax[0].plot(current_agent.solt, err_pose[:,0], label="x error", linewidth=3, color=colorlist[0])
# ax[0].plot(current_agent.solt, err_pose[:,1], label="y error", linewidth=3, color=colorlist[1])
# ax[0].plot(current_agent.solt, err_pose[:,2], label="z error", linewidth=3, color=colorlist[2])
# ax[0].legend(markerscale=15, loc="upper right", fontsize=30) #, bbox_to_anchor=(0.65, 0.85))
# ax[0].set_xlabel("Time (sec)",  fontsize=24)
# ax[0].set_ylabel(ylabel[0],     fontsize=34)
# ax[0].grid()
# ax[0].set_xlim(0, 30)
# ax[0].set_ylim(-5.5, 5.5)
# ax[0].minorticks_on()

# ax[0].tick_params(axis='both', labelsize=28)
# ax[1].tick_params(axis='both', labelsize=28)
# ax[0].yaxis.set_label_coords(-0.12, 0.5)
# ax[1].yaxis.set_label_coords(-0.08, 0)
# # ax[0,1].plot(self.solt, np.array(plot_err_lambda_bar)[:,0:3], label=["x", "y", "z"], marker='o', markersize=0.2, color=colorlist)
# ax[1].plot(current_agent.solt, np.array(plt_err_degree)[:,0], label="roll error", linewidth=3, color=colorlist[0])
# ax[1].plot(current_agent.solt, np.array(plt_err_degree)[:,1], label="pitch error", linewidth=3, color=colorlist[1])
# ax[1].plot(current_agent.solt, np.array(plt_err_degree)[:,2], label="yaw error", linewidth=3, color=colorlist[2])
# ax[1].legend(markerscale=15, loc="upper right", fontsize=28) #, bbox_to_anchor=(0.65, 0.9))
# ax[1].set_xlabel("Time (sec)",  fontsize=24)
# ax[1].set_ylabel(ylabel[1],     fontsize=34)
# ax[1].grid()
# ax[1].set_xlim(0, 30)
# ax[1].set_ylim(-100, 90)
# ax[1].minorticks_on()

# figure.suptitle(f"{name}", fontsize=50)
    


In [ ]:

# ax3d = plt.figure(figsize=(14,14)).add_subplot(projection='3d')
# color_est = ["tomato", "mediumseagreen", "dodgerblue"]
# ms_size = 20
# ax3d.plot(np.array(agent_1_plot_est_p)[:,0], np.array(agent_1_plot_est_p)[:,1], np.array(agent_1_plot_est_p)[:,2], color=color_est[0], label=r'Vehicle $f_1$ - Estimation')
# ax3d.plot(np.array(agent_1_plot_est_p)[0,0], np.array(agent_1_plot_est_p)[0,1], np.array(agent_1_plot_est_p)[0,2], color=color_est[0], ms=ms_size, marker="^")
# ax3d.plot(np.array(agent_1_plot_est_p)[-1,0], np.array(agent_1_plot_est_p)[-1,1], np.array(agent_1_plot_est_p)[-1,2], color=color_est[0], ms=ms_size, marker="X")
# ax3d.plot(np.array(agent_1_plot_act_p)[:,0], np.array(agent_1_plot_act_p)[:,1], np.array(agent_1_plot_act_p)[:,2], color=color_est[0], label=r'Vehicle $f_1$ - Ground Truth', linestyle="--")
# ax3d.plot(np.array(agent_1_plot_act_p)[0,0], np.array(agent_1_plot_act_p)[0,1], np.array(agent_1_plot_act_p)[0,2], color=color_est[0], ms=ms_size, marker="^")
# ax3d.plot(np.array(agent_1_plot_act_p)[-1,0], np.array(agent_1_plot_act_p)[-1,1], np.array(agent_1_plot_act_p)[-1,2], color=color_est[0], ms=ms_size, marker="X")

# ax3d.plot(np.array(agent_2_plot_est_p)[:,0], np.array(agent_2_plot_est_p)[:,1], np.array(agent_2_plot_est_p)[:,2], color=color_est[1], label=r'Vehicle $f_2$ - Estimation')
# ax3d.plot(np.array(agent_2_plot_est_p)[0,0], np.array(agent_2_plot_est_p)[0,1], np.array(agent_2_plot_est_p)[0,2], color=color_est[1], ms=ms_size, marker="^")
# ax3d.plot(np.array(agent_2_plot_est_p)[-1,0], np.array(agent_2_plot_est_p)[-1,1], np.array(agent_2_plot_est_p)[-1,2], color=color_est[1], ms=ms_size, marker="X")
# ax3d.plot(np.array(agent_2_plot_act_p)[:,0], np.array(agent_2_plot_act_p)[:,1], np.array(agent_2_plot_act_p)[:,2], color=color_est[1], label=r'Vehicle $f_2$ - Ground Truth', linestyle="--")
# ax3d.plot(np.array(agent_2_plot_act_p)[0,0], np.array(agent_2_plot_act_p)[0,1], np.array(agent_2_plot_act_p)[0,2], color=color_est[1], ms=ms_size, marker="^")
# ax3d.plot(np.array(agent_2_plot_act_p)[-1,0], np.array(agent_2_plot_act_p)[-1,1], np.array(agent_2_plot_act_p)[-1,2], color=color_est[1], ms=ms_size, marker="X")

# ax3d.plot(np.array(agent_3_plot_est_p)[:,0], np.array(agent_3_plot_est_p)[:,1], np.array(agent_3_plot_est_p)[:,2], color=color_est[2], label=r'Vehicle $f_3$ - Estimation')
# ax3d.plot(np.array(agent_3_plot_est_p)[0,0], np.array(agent_3_plot_est_p)[0,1], np.array(agent_3_plot_est_p)[0,2], color=color_est[2], ms=ms_size, marker="^")
# ax3d.plot(np.array(agent_3_plot_est_p)[-1,0], np.array(agent_3_plot_est_p)[-1,1], np.array(agent_3_plot_est_p)[-1,2], color=color_est[2], ms=ms_size, marker="X")
# ax3d.plot(np.array(agent_3_plot_act_p)[:,0], np.array(agent_3_plot_act_p)[:,1], np.array(agent_3_plot_act_p)[:,2], color=color_est[2], label=r'Vehicle $f_3$ - Ground Truth', linestyle="--")
# ax3d.plot(np.array(agent_3_plot_act_p)[0,0], np.array(agent_3_plot_act_p)[0,1], np.array(agent_3_plot_act_p)[0,2], color=color_est[2], ms=ms_size, marker="^")
# ax3d.plot(np.array(agent_3_plot_act_p)[-1,0], np.array(agent_3_plot_act_p)[-1,1], np.array(agent_3_plot_act_p)[-1,2], color=color_est[2], ms=ms_size, marker="X")

# landmark_size = 200
# ax3d.scatter(0, 0, 0, color='darkblue',  s=landmark_size, marker="o")
# ax3d.scatter(5, -2, 0, color='darkblue', s=landmark_size, marker="o")
# ax3d.scatter(3, 3, 3, color='darkblue',  s=landmark_size, marker="o")
# ax3d.legend(markerscale=25, loc="upper right", fontsize=20)
# ax3d.set_xlabel("x (m)", fontsize=18)
# ax3d.set_ylabel("y (m)", fontsize=18)
# ax3d.set_zlabel("z (m)", fontsize=18)
# ax3d.tick_params(labelsize=18)

In [ ]:
def cascade_intersection_setting(first_agent=None, second_agent=None, third_agent=None, fourth_agent=None, save_fig=False, need_y_lim=False, time=(0,1)):
    print("agent1")
    if first_agent == None:

        p_formula = lambda t: np.transpose(np.array([[-2,-16 + 0.6 * t,2]]))
        v_formula = lambda t: np.transpose(np.array([[0, 0.6, 0]]))
        omega_formula = lambda t: np.transpose(np.array([[0, 0, 0]]))

        use_adaptive            = True
        quaternion              = True
        stepsize                = 0.0001
        tol                     = 1e-3
        noise                   = False
        which_eq                = 0
        which_omega             = "full" # "z" or "full"
        with_image_hz_sim       = False
        image_hz                = 60
        randomize_image_input   = False
        p_hat                   = np.array([[0, -5, 5]], dtype=np.float64).T
        qua = [0.7071068, 0, 0, 0.7071068]
        qua /= np.linalg.norm(qua)
        Lambda_bar_0            = np.array(qua, dtype=np.float64).T  # quaternion: w, x, y, z
        z_appear                = lambda t: ([[[-4,5,3]],[[4,4,5]],[[4,-3,4]]])
        z_appear_act            = z_appear
        l                       = 3
        k                       = 1
        q                       = [10]
        p_ricatti               = np.array([1, 100])
        first_name = "Follower 1"


        agent_first_follower = riccati_observer(
            use_adaptive            = use_adaptive,
            quaternion              = quaternion,
            p_formula               = p_formula,
            omega_formula           = omega_formula,
            v_formula               = v_formula,
            time                    = time,
            stepsize                = stepsize,
            tol                     = tol,
            noise                   = noise,
            which_eq                = which_eq,
            which_omega             = which_omega,
            with_image_hz_sim       = with_image_hz_sim, 
            image_hz                = image_hz, 
            randomize_image_input   = randomize_image_input, 
            p_hat                   = p_hat,
            Lambda_bar_0            = Lambda_bar_0,  # quaternion: w, x, y, z
            z_appear                = z_appear,
            z_appear_act            = z_appear_act,
            l                       = l,
            k                       = k,
            q                       = q, 
            v                       = np.array([0.1, 1]),
            p_ricatti               = p_ricatti,
            name                    = first_name

            )
        
        first_solt, first_soly, first_caltime, first_args = agent_first_follower.full_simulation()

        ############### INTERPOLATION ###############
        first_est_p = []
        agent_first_solution = np.array(first_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(first_solt, first_soly)):
            if first_agent == None:
                own_sol_est_R = agent_first_follower.rodrigues_formula(solution[4:8])
            else:
                own_sol_est_R = first_agent.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            first_est_p.append(p_temp)
        first_est_p = np.array(first_est_p)
        first_agent_interpolation = CubicSpline(first_solt, first_est_p)
        ############### SECOND FOLLOWER ###############

    else:
        first_solt = first_agent.solt
        first_soly = first_agent.soly
        ############### INTERPOLATION ###############
        first_est_p = []
        agent_first_solution = np.array(first_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(first_solt, first_soly)):
            if first_agent == None:
                own_sol_est_R = first_agent.rodrigues_formula(solution[4:8])
            else:
                own_sol_est_R = first_agent.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            first_est_p.append(p_temp)
        first_est_p = np.array(first_est_p)
        first_agent_interpolation = CubicSpline(first_solt, first_est_p)
        ############### SECOND FOLLOWER ###############


    print("agent2")
    if second_agent == None:
        second_p_formula = lambda t: np.transpose(np.array([[-2,-19+0.8*(t),2]]))
        second_v_formula = lambda t: np.transpose(np.array([[0, 0.8, 0]]))
        second_omega_formula = lambda t: np.transpose(np.array([[0, 0, 0]]))

        second_use_adaptive            = True
        second_quaternion              = True
        second_stepsize                = 0.001
        second_tol                     = 1e-3
        second_noise                   = False
        second_which_eq                = 0
        second_which_omega             = "full" # "z" or "full"
        second_with_image_hz_sim       = False
        second_image_hz                = 60
        second_randomize_image_input   = False
        second_p_hat                   = np.array([[5, -14, 6]], dtype=np.float64).T
        second_qua = [0.7071068, 0, 0, 0.7071068]
        second_qua /= np.linalg.norm(second_qua)
        second_Lambda_bar_0            = np.array(second_qua, dtype=np.float64).T  # quaternion: w, x, y, z
        second_z_appear                = lambda t: ([[[4,4,5]],[[4,-3,4]], [first_agent_interpolation(t)]])
        second_z_appear_act            = lambda t: ([[[4,4,5]],[[4,-3,4]], [[-2,-16 + 0.6 * t,2]]])
        second_l                       = 3
        second_k                       = 1
        second_q                       = [10]
        second_p_ricatti               = np.array([1, 100])
        second_name = "Follower 2"

        agent_second_follower = riccati_observer(
            use_adaptive            = second_use_adaptive,
            quaternion              = second_quaternion,
            p_formula               = second_p_formula,
            omega_formula           = second_omega_formula,
            v_formula               = second_v_formula,
            time                    = time,
            stepsize                = second_stepsize,
            tol                     = second_tol,
            noise                   = second_noise,
            which_eq                = second_which_eq,
            which_omega             = second_which_omega,
            with_image_hz_sim       = second_with_image_hz_sim, 
            image_hz                = second_image_hz, 
            randomize_image_input   = second_randomize_image_input, 
            p_hat                   = second_p_hat,
            Lambda_bar_0            = second_Lambda_bar_0,  # quaternion: w, x, y, z
            z_appear                = second_z_appear,
            z_appear_act            = second_z_appear_act,
            l                       = second_l,
            k                       = second_k,
            q                       = second_q, 
            v                       = np.array([0.1, 1]),
            p_ricatti               = second_p_ricatti,
            name                    = second_name
            )
        
        second_solt, second_soly, second_caltime, second_args = agent_second_follower.full_simulation()
        ############### INTERPOLATION ###############
        second_est_p = []
        agent_second_solution = np.array(second_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(second_solt, second_soly)):
            own_sol_est_R = agent_second_follower.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            second_est_p.append(p_temp)
        second_est_p = np.array(second_est_p)
        second_agent_interpolation = CubicSpline(second_solt, second_est_p)
        ############### THIRD FOLLOWER ###############
    else:
        second_solt = second_agent.solt
        second_soly = second_agent.soly
        ############### INTERPOLATION ###############
        second_est_p = []
        agent_second_solution = np.array(second_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(second_solt, second_soly)):
            own_sol_est_R = second_agent.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            second_est_p.append(p_temp)
        second_est_p = np.array(second_est_p)
        second_agent_interpolation = CubicSpline(second_solt, second_est_p)
        ############### THIRD FOLLOWER ###############


    print("agent3")
    if third_agent == None:
        third_p_formula = lambda t: np.transpose(np.array([[-17+0.7*(t),2,3]]))
        third_v_formula = lambda t: np.transpose(np.array([[0.7, 0, 0]]))
        third_omega_formula = lambda t: np.transpose(np.array([[0, 0, 0]]))

        third_use_adaptive            = True
        third_quaternion              = True
        third_stepsize                = 0.001
        third_tol                     = 1e-3
        third_noise                   = False
        third_which_eq                = 0
        third_which_omega             = "full" # "z" or "full"
        third_with_image_hz_sim       = False
        third_image_hz                = 60
        third_randomize_image_input   = False
        third_p_hat                   = np.array([[-8, 3, 5]], dtype=np.float64).T
        third_qua = [0.7071068, 0, 0, 0.7071068]
        third_qua /= np.linalg.norm(third_qua)
        third_Lambda_bar_0            = np.array(third_qua, dtype=np.float64).T  # quaternion: w, x, y, z
        third_z_appear                = lambda t: ([[[4,4,5]], [[4,-3,4]], [second_agent_interpolation(t)]])
        third_z_appear_act            = lambda t: ([[[4,4,5]], [[4,-3,4]], [[-2,-19+0.8*(t),2]]])
        third_l                       = 3
        third_k                       = 1
        third_q                       = [10]
        third_p_ricatti               = np.array([1, 100])
        third_name = "Follower 3"


        agent_third_follower = riccati_observer(
            use_adaptive            = third_use_adaptive,
            quaternion              = third_quaternion,
            p_formula               = third_p_formula,
            omega_formula           = third_omega_formula,
            v_formula               = third_v_formula,
            time                    = time,
            stepsize                = third_stepsize,
            tol                     = third_tol,
            noise                   = third_noise,
            which_eq                = third_which_eq,
            which_omega             = third_which_omega,
            with_image_hz_sim       = third_with_image_hz_sim, 
            image_hz                = third_image_hz, 
            randomize_image_input   = third_randomize_image_input, 
            p_hat                   = third_p_hat,
            Lambda_bar_0            = third_Lambda_bar_0,  # quaternion: w, x, y, z
            z_appear                = third_z_appear,
            z_appear_act            = third_z_appear_act,
            l                       = third_l,
            k                       = third_k,
            q                       = third_q, 
            v                       = np.array([0.1, 1]),
            p_ricatti               = third_p_ricatti,
            name                    = third_name
            )
            
        third_solt, third_soly, third_caltime, third_args = agent_third_follower.full_simulation()
        ############### INTERPOLATION ###############
        third_est_p = []
        agent_third_solution = np.array(third_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(third_solt, third_soly)):
            own_sol_est_R = agent_third_follower.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            third_est_p.append(p_temp)
        third_est_p = np.array(third_est_p)
        third_agent_interpolation = CubicSpline(third_solt, third_est_p)
        ############### THIRD FOLLOWER ###############
    else:
        third_solt = third_agent.solt
        third_soly = third_agent.soly
        ############### INTERPOLATION ###############
        third_est_p = []
        agent_third_solution = np.array(third_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(third_solt, third_soly)):
            own_sol_est_R = third_agent.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            third_est_p.append(p_temp)
        third_est_p = np.array(third_est_p)
        third_agent_interpolation = CubicSpline(third_solt, third_est_p)
        ############### THIRD FOLLOWER ###############


    print("agent4")
    if fourth_agent == None:
        fourth_p_formula = lambda t: np.transpose(np.array([[-19+0.5*(t),2,3]]))
        fourth_v_formula = lambda t: np.transpose(np.array([[0.5, 0, 0]]))
        fourth_omega_formula = lambda t: np.transpose(np.array([[0, 0, 0]]))

        fourth_use_adaptive            = True
        fourth_quaternion              = True
        fourth_stepsize                = 0.001
        fourth_tol                     = 1e-3
        fourth_noise                   = False
        fourth_which_eq                = 0
        fourth_which_omega             = "full" # "z" or "full"
        fourth_with_image_hz_sim       = False
        fourth_image_hz                = 60
        fourth_randomize_image_input   = False
        fourth_p_hat                   = np.array([[-14, 6, 6]], dtype=np.float64).T
        fourth_qua = [0.7071068, 0, 0, 0.7071068]
        fourth_qua /= np.linalg.norm(fourth_qua)
        fourth_Lambda_bar_0            = np.array(fourth_qua, dtype=np.float64).T  # quaternion: w, x, y, z
        fourth_z_appear                = lambda t: ([[[4,-3, 4]], [second_agent_interpolation(t)], [third_agent_interpolation(t)]])
        fourth_z_appear_act            = lambda t: ([[[4,-3, 4]], [[-2,-19+0.8*(t),2]], [[-17+0.7*(t),2,3]]])
        fourth_l                       = 3
        fourth_k                       = 1
        fourth_q                       = [10]
        fourth_p_ricatti               = np.array([1, 100])
        fourth_name = "Follower 4"


        agent_fourth_follower = riccati_observer(
            use_adaptive            = fourth_use_adaptive,
            quaternion              = fourth_quaternion,
            p_formula               = fourth_p_formula,
            omega_formula           = fourth_omega_formula,
            v_formula               = fourth_v_formula,
            time                    = time,
            stepsize                = fourth_stepsize,
            tol                     = fourth_tol,
            noise                   = fourth_noise,
            which_eq                = fourth_which_eq,
            which_omega             = fourth_which_omega,
            with_image_hz_sim       = fourth_with_image_hz_sim, 
            image_hz                = fourth_image_hz, 
            randomize_image_input   = fourth_randomize_image_input, 
            p_hat                   = fourth_p_hat,
            Lambda_bar_0            = fourth_Lambda_bar_0,  # quaternion: w, x, y, z
            z_appear                = fourth_z_appear,
            z_appear_act            = fourth_z_appear_act,
            l                       = fourth_l,
            k                       = fourth_k,
            q                       = fourth_q, 
            v                       = np.array([0.1, 1]),
            p_ricatti               = fourth_p_ricatti,
            name                    = fourth_name
            )
            
        fourth_solt, fourth_soly, fourth_caltime, fourth_args = agent_fourth_follower.full_simulation()
        ############### INTERPOLATION ###############
        fourth_est_p = []
        agent_fourth_solution = np.array(fourth_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(fourth_solt, fourth_soly)):
            own_sol_est_R = agent_fourth_follower.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            fourth_est_p.append(p_temp)
        fourth_est_p = np.array(fourth_est_p)
        fourth_agent_interpolation = CubicSpline(fourth_solt, fourth_est_p)
        ############### fourth FOLLOWER ###############
    else:
        fourth_solt = fourth_agent.solt
        fourth_soly = fourth_agent.soly
        ############### INTERPOLATION ###############
        fourth_est_p = []
        agent_fourth_solution = np.array(fourth_soly)[:, 4:11]
        for idx, (t, solution) in enumerate(zip(fourth_solt, fourth_soly)):
            own_sol_est_R = fourth_agent.rodrigues_formula(solution[4:8])
            own_sol_est_p_bar = solution[8:11]
            p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)
            fourth_est_p.append(p_temp)
        fourth_est_p = np.array(fourth_est_p)
        fourth_agent_interpolation = CubicSpline(fourth_solt, fourth_est_p)
        ############### fourth FOLLOWER ###############


    print("agent5")
    fifth_p_formula = lambda t: np.transpose(np.array([[-30+0.6*(t),2,2]]))
    fifth_v_formula = lambda t: np.transpose(np.array([[0.6, 0, 0]]))
    fifth_omega_formula = lambda t: np.transpose(np.array([[0, 0, 0]]))

    fifth_use_adaptive            = True
    fifth_quaternion              = True
    fifth_stepsize                = 0.001
    fifth_tol                     = 1e-3
    fifth_noise                   = False
    fifth_which_eq                = 0
    fifth_which_omega             = "full" # "z" or "full"
    fifth_with_image_hz_sim       = False
    fifth_image_hz                = 60
    fifth_randomize_image_input   = False
    fifth_p_hat                   = np.array([[-24, 7, 6]], dtype=np.float64).T
    fifth_qua = [0.7071068, 0, 0, 0.7071068]
    fifth_qua /= np.linalg.norm(fifth_qua)
    fifth_Lambda_bar_0            = np.array(fifth_qua, dtype=np.float64).T  # quaternion: w, x, y, z
    fifth_z_appear                = lambda t: ([[first_agent_interpolation(t)], [second_agent_interpolation(t)], [fourth_agent_interpolation(t)]])
    fifth_z_appear_act            = lambda t: ([[[-2,-16 + 0.6 * t,2]], [[-2,-19+0.8*(t),2]], [[-19+0.5*(t),2,3]]])
    fifth_l                       = 3
    fifth_k                       = 1
    fifth_q                       = [10]
    fifth_p_ricatti               = np.array([1, 100])
    fifth_name = "Follower 5"


    agent_fifth_follower = riccati_observer(
        use_adaptive            = fifth_use_adaptive,
        quaternion              = fifth_quaternion,
        p_formula               = fifth_p_formula,
        omega_formula           = fifth_omega_formula,
        v_formula               = fifth_v_formula,
        time                    = time,
        stepsize                = fifth_stepsize,
        tol                     = fifth_tol,
        noise                   = fifth_noise,
        which_eq                = fifth_which_eq,
        which_omega             = fifth_which_omega,
        with_image_hz_sim       = fifth_with_image_hz_sim, 
        image_hz                = fifth_image_hz, 
        randomize_image_input   = fifth_randomize_image_input, 
        p_hat                   = fifth_p_hat,
        Lambda_bar_0            = fifth_Lambda_bar_0,  # quaternion: w, x, y, z
        z_appear                = fifth_z_appear,
        z_appear_act            = fifth_z_appear_act,
        l                       = fifth_l,
        k                       = fifth_k,
        q                       = fifth_q, 
        v                       = np.array([0.1, 1]),
        p_ricatti               = fifth_p_ricatti,
        name                    = fifth_name
        )
        
    fifth_solt, fifth_soly, fifth_caltime, fifth_args = agent_fifth_follower.full_simulation()



    print("Plotting")
    if first_agent == None:
        fig, ax, lambda_, p_bar = agent_first_follower.visual_plot(figsize=(12,12), bound_y=True) 
    else:
        fig, ax, lambda_, p_bar = first_agent.visual_plot(figsize=(12,12), bound_y=True)  
        agent_first_follower = first_agent 

    if second_agent == None:
        ig, ax, lambda_, p_bar = agent_second_follower.visual_plot(figsize=(12,12), bound_y=True)  
    else:
        fig, ax, lambda_, p_bar = second_agent.visual_plot(figsize=(12,12), bound_y=True)  
        agent_second_follower = second_agent
    if third_agent == None:
        ig, ax, lambda_, p_bar = agent_third_follower.visual_plot(figsize=(12,12), bound_y=True)  
    else:
        fig, ax, lambda_, p_bar = third_agent.visual_plot(figsize=(12,12), bound_y=True)  
        agent_third_follower = third_agent
    
    if fourth_agent == None:
        ig, ax, lambda_, p_bar = agent_fourth_follower.visual_plot(figsize=(12,12), bound_y=True)  
    else:
        fig, ax, lambda_, p_bar = fourth_agent.visual_plot(figsize=(12,12), bound_y=True)  
        agent_fourth_follower = fourth_agent

    ig, ax, lambda_, p_bar = agent_fifth_follower.visual_plot(figsize=(12,12), bound_y=True)  


    return (agent_first_follower, agent_second_follower, agent_third_follower, agent_fourth_follower, agent_fifth_follower)


In [ ]:
cascade_intersection_agents = cascade_intersection_setting(time=(0, 50.5)) #first_agent=cascade_intersection_agents[0], second_agent=cascade_intersection_agents[1], 

In [ ]:
agent_1 = cascade_intersection_agents[0]
agent_2 = cascade_intersection_agents[1]
agent_3 = cascade_intersection_agents[2]
agent_4 = cascade_intersection_agents[3]
agent_5 = cascade_intersection_agents[4]

agent_1_plot_est_p = []
agent_1_plot_act_p = []
agent_1_plt_err_degree = []
for idx, (t, solution) in enumerate(zip(agent_1.solt, agent_1.soly)):
    own_sol_R = agent_1.rodrigues_formula(solution[:4])
    own_sol_est_R = agent_1.rodrigues_formula(solution[4:8])
    own_sol_est_p_bar = solution[8:11]
    p = agent_1.GetPose(t).reshape((3,))
    # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
    p_bar_temp = np.matmul(np.transpose(np.array(own_sol_R).reshape((3,3))), p)
    p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)

    agent_1_plot_est_p.append(p_temp.tolist())
    agent_1_plot_act_p.append(p.tolist())

    err = np.matmul(np.linalg.inv(np.array(own_sol_est_R).reshape((3,3))), np.array(own_sol_R).reshape((3,3)))
    err = ScipyRot.from_matrix(err)
    err_degree = err.as_euler('xyz', degrees=True)
    agent_1_plt_err_degree.append(err_degree)

agent_2_plot_est_p = []
agent_2_plot_act_p = []
agent_2_plt_err_degree = []
for idx, (t, solution) in enumerate(zip(agent_2.solt, agent_2.soly)):
    own_sol_R = agent_2.rodrigues_formula(solution[:4])
    own_sol_est_R = agent_2.rodrigues_formula(solution[4:8])
    own_sol_est_p_bar = solution[8:11]
    p = agent_2.GetPose(t).reshape((3,))
    # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
    p_bar_temp = np.matmul(np.transpose(np.array(own_sol_R).reshape((3,3))), p)
    p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)

    agent_2_plot_est_p.append(p_temp.tolist())
    agent_2_plot_act_p.append(p.tolist())

    err = np.matmul(np.linalg.inv(np.array(own_sol_est_R).reshape((3,3))), np.array(own_sol_R).reshape((3,3)))
    err = ScipyRot.from_matrix(err)
    err_degree = err.as_euler('xyz', degrees=True)
    agent_2_plt_err_degree.append(err_degree)

agent_3_plot_est_p = []
agent_3_plot_act_p = []
agent_3_plt_err_degree = []
for idx, (t, solution) in enumerate(zip(agent_3.solt, agent_3.soly)):
    own_sol_R = agent_3.rodrigues_formula(solution[:4])
    own_sol_est_R = agent_3.rodrigues_formula(solution[4:8])
    own_sol_est_p_bar = solution[8:11]
    p = agent_3.GetPose(t).reshape((3,))
    # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
    p_bar_temp = np.matmul(np.transpose(np.array(own_sol_R).reshape((3,3))), p)
    p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)

    agent_3_plot_est_p.append(p_temp.tolist())
    agent_3_plot_act_p.append(p.tolist())

    err = np.matmul(np.linalg.inv(np.array(own_sol_est_R).reshape((3,3))), np.array(own_sol_R).reshape((3,3)))
    err = ScipyRot.from_matrix(err)
    err_degree = err.as_euler('xyz', degrees=True)
    agent_3_plt_err_degree.append(err_degree)

agent_4_plot_est_p = []
agent_4_plot_act_p = []
agent_4_plt_err_degree = []
for idx, (t, solution) in enumerate(zip(agent_4.solt, agent_4.soly)):
    own_sol_R = agent_4.rodrigues_formula(solution[:4])
    own_sol_est_R = agent_4.rodrigues_formula(solution[4:8])
    own_sol_est_p_bar = solution[8:11]
    p = agent_4.GetPose(t).reshape((3,))
    # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
    p_bar_temp = np.matmul(np.transpose(np.array(own_sol_R).reshape((3,3))), p)
    p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)

    agent_4_plot_est_p.append(p_temp.tolist())
    agent_4_plot_act_p.append(p.tolist())

    err = np.matmul(np.linalg.inv(np.array(own_sol_est_R).reshape((3,3))), np.array(own_sol_R).reshape((3,3)))
    err = ScipyRot.from_matrix(err)
    err_degree = err.as_euler('xyz', degrees=True)
    agent_4_plt_err_degree.append(err_degree)


agent_5_plot_est_p = []
agent_5_plot_act_p = []
agent_5_plt_err_degree = []
for idx, (t, solution) in enumerate(zip(agent_5.solt, agent_5.soly)):
    own_sol_R = agent_5.rodrigues_formula(solution[:4])
    own_sol_est_R = agent_5.rodrigues_formula(solution[4:8])
    own_sol_est_p_bar = solution[8:11]
    p = agent_5.GetPose(t).reshape((3,))
    # p = np.array([2.5+2.5*np.cos(0.4*t), 2.5*np.sin(0.4*t), 10])
    p_bar_temp = np.matmul(np.transpose(np.array(own_sol_R).reshape((3,3))), p)
    p_temp = np.matmul(np.array(own_sol_est_R).reshape((3,3)), own_sol_est_p_bar)

    agent_5_plot_est_p.append(p_temp.tolist())
    agent_5_plot_act_p.append(p.tolist())

    err = np.matmul(np.linalg.inv(np.array(own_sol_est_R).reshape((3,3))), np.array(own_sol_R).reshape((3,3)))
    err = ScipyRot.from_matrix(err)
    err_degree = err.as_euler('xyz', degrees=True)
    agent_5_plt_err_degree.append(err_degree)

In [ ]:
%matplotlib widget
current_agents =     [agent_1, agent_2, agent_3, agent_4, agent_5]
plot_act_p =        [agent_1_plot_act_p, agent_2_plot_act_p, agent_3_plot_act_p, agent_4_plot_act_p, agent_5_plot_act_p]
plot_est_p =        [agent_1_plot_est_p, agent_2_plot_est_p, agent_3_plot_est_p, agent_4_plot_est_p, agent_5_plot_est_p]
plt_err_degree =    [agent_1_plt_err_degree, agent_2_plt_err_degree, agent_3_plt_err_degree, agent_4_plt_err_degree, agent_5_plt_err_degree]

figure, ax = plt.subplots(5,2, figsize=(16,16))
colorlist = ["tomato", "limegreen", "slateblue"]
for current_agent in current_agents:
    print("hiii")
    if "1" in current_agent.name:
        agentidx = 0
        name = r"Vehicle $f_1$"
        ylabel = [r'$\tilde{\bar p}_{f_1}$ (m)' , r'deg']
    elif "2" in current_agent.name:
        agentidx = 1
        name = r"Vehicle $f_2$"
        ylabel = [r'$\tilde{\bar p}_{f_2}$ (m)' , r'deg']
    elif "3" in current_agent.name:
        agentidx = 2
        name = r"Vehicle $f_3$"
        ylabel = [r'$\tilde{\bar p}_{f_3}$ (m)' , r'deg']
    elif "4" in current_agent.name:
        agentidx = 3
        name = r"Vehicle $f_4$"
        ylabel = [r'$\tilde{\bar p}_{f_4}$ (m)' , r'deg']
    elif "5" in current_agent.name:
        agentidx = 4
        name = r"Vehicle $f_5$"
        ylabel = [r'$\tilde{\bar p}_{f_5}$ (m)' , r'deg']

    err_pose = np.array(plot_act_p[agentidx])-np.array(plot_est_p[agentidx])
    ax[agentidx,0].plot(current_agent.solt, err_pose[:,0], label="x error", linewidth=3, color=colorlist[0])
    ax[agentidx,0].plot(current_agent.solt, err_pose[:,1], label="y error", linewidth=3, color=colorlist[1])
    ax[agentidx,0].plot(current_agent.solt, err_pose[:,2], label="z error", linewidth=3, color=colorlist[2])
    ax[agentidx,0].set_ylabel(ylabel[0],     fontsize=34)
    ax[agentidx,0].grid()
    ax[agentidx,0].set_xlim(0, 50)
    ax[agentidx,0].set_ylim(-0.5, 0.5)
    ax[agentidx,0].minorticks_on()
    if agentidx != 4:
        ax[agentidx, 0].set_xticklabels([])

    ax[agentidx, 0].tick_params(axis='both', labelsize=28)
    ax[agentidx, 0].yaxis.set_label_coords(-0.12, 0.5)
    ax[agentidx, 1].tick_params(axis='both', labelsize=28)
    ax[agentidx, 1].yaxis.set_label_coords(-0.08, 0)
    # ax[0,1].plot(self.solt, np.array(plot_err_lambda_bar)[:,0:3], label=["x", "y", "z"], marker='o', markersize=0.2, color=colorlist)
    ax[agentidx, 1].plot(current_agent.solt, np.array(plt_err_degree[agentidx])[:,0], label="roll error", linewidth=3, color=colorlist[0])
    ax[agentidx, 1].plot(current_agent.solt, np.array(plt_err_degree[agentidx])[:,1], label="pitch error", linewidth=3, color=colorlist[1])
    ax[agentidx, 1].plot(current_agent.solt, np.array(plt_err_degree[agentidx])[:,2], label="yaw error", linewidth=3, color=colorlist[2])
    ax[agentidx, 1].set_ylabel(ylabel[1],     fontsize=34)
    ax[agentidx, 1].grid()
    ax[agentidx, 1].set_xlim(0, 50)
    ax[agentidx, 1].set_ylim(-100, 90)
    ax[agentidx, 1].minorticks_on()
    if agentidx != 4:
        ax[agentidx, 1].set_xticklabels([])

    ax[4, 0].set_xlabel("Time (sec)",  fontsize=24)
    ax[4, 1].set_xlabel("Time (sec)",  fontsize=24)
    ax[0, 0].legend(markerscale=15, loc="center left", bbox_to_anchor=(0.5,  1.05), fontsize=30) #
    ax[0, 1].legend(markerscale=15, loc="center left", bbox_to_anchor=(0.35, 1.05), fontsize=28) #

figure.suptitle("Simulation of a busy intersection scenario", fontsize=30)
    


In [ ]:

ax3d = plt.figure(figsize=(10, 10)).add_subplot(projection='3d')
color_est = ["tomato", "orange", "mediumseagreen", "dodgerblue", "orchid"]
ms_size = 15
ax3d.plot(np.array(agent_1_plot_est_p)[:,0], np.array(agent_1_plot_est_p)[:,1], np.array(agent_1_plot_est_p)[:,2], color=color_est[0], label=r'Vehicle $f_1$')
ax3d.plot(np.array(agent_1_plot_est_p)[0,0], np.array(agent_1_plot_est_p)[0,1], np.array(agent_1_plot_est_p)[0,2], color=color_est[0], ms=ms_size, marker="^")
ax3d.plot(np.array(agent_1_plot_est_p)[-1,0], np.array(agent_1_plot_est_p)[-1,1], np.array(agent_1_plot_est_p)[-1,2], color=color_est[0], ms=ms_size, marker="X")
ax3d.plot(np.array(agent_1_plot_act_p)[:,0], np.array(agent_1_plot_act_p)[:,1], np.array(agent_1_plot_act_p)[:,2], color=color_est[0], linestyle="--")
# ax3d.plot(np.array(agent_1_plot_act_p)[:,0], np.array(agent_1_plot_act_p)[:,1], np.array(agent_1_plot_act_p)[:,2], color=color_est[0], label=r'Vehicle $f_1$ - Ground Truth', linestyle="--")
ax3d.plot(np.array(agent_1_plot_act_p)[0,0], np.array(agent_1_plot_act_p)[0,1], np.array(agent_1_plot_act_p)[0,2], color=color_est[0], ms=ms_size, marker="^", fillstyle="none")
ax3d.plot(np.array(agent_1_plot_act_p)[-1,0], np.array(agent_1_plot_act_p)[-1,1], np.array(agent_1_plot_act_p)[-1,2], color=color_est[0], ms=ms_size, marker="X", fillstyle="none")

ax3d.plot(np.array(agent_2_plot_est_p)[:,0], np.array(agent_2_plot_est_p)[:,1], np.array(agent_2_plot_est_p)[:,2], color=color_est[1], label=r'Vehicle $f_2$')
ax3d.plot(np.array(agent_2_plot_est_p)[0,0], np.array(agent_2_plot_est_p)[0,1], np.array(agent_2_plot_est_p)[0,2], color=color_est[1], ms=ms_size, marker="^")
ax3d.plot(np.array(agent_2_plot_est_p)[-1,0], np.array(agent_2_plot_est_p)[-1,1], np.array(agent_2_plot_est_p)[-1,2], color=color_est[1], ms=ms_size, marker="X")
ax3d.plot(np.array(agent_2_plot_act_p)[:,0], np.array(agent_2_plot_act_p)[:,1], np.array(agent_2_plot_act_p)[:,2], color=color_est[1], linestyle="--")
# ax3d.plot(np.array(agent_2_plot_act_p)[:,0], np.array(agent_2_plot_act_p)[:,1], np.array(agent_2_plot_act_p)[:,2], color=color_est[1], label=r'Vehicle $f_2$ - Ground Truth', linestyle="--")
ax3d.plot(np.array(agent_2_plot_act_p)[0,0], np.array(agent_2_plot_act_p)[0,1], np.array(agent_2_plot_act_p)[0,2], color=color_est[1], ms=ms_size, marker="^", fillstyle="none")
ax3d.plot(np.array(agent_2_plot_act_p)[-1,0], np.array(agent_2_plot_act_p)[-1,1], np.array(agent_2_plot_act_p)[-1,2], color=color_est[1], ms=ms_size, marker="X", fillstyle="none")

ax3d.plot(np.array(agent_3_plot_est_p)[:,0], np.array(agent_3_plot_est_p)[:,1], np.array(agent_3_plot_est_p)[:,2], color=color_est[2], label=r'Vehicle $f_3$')
ax3d.plot(np.array(agent_3_plot_est_p)[0,0], np.array(agent_3_plot_est_p)[0,1], np.array(agent_3_plot_est_p)[0,2], color=color_est[2], ms=ms_size, marker="^")
ax3d.plot(np.array(agent_3_plot_est_p)[-1,0], np.array(agent_3_plot_est_p)[-1,1], np.array(agent_3_plot_est_p)[-1,2], color=color_est[2], ms=ms_size, marker="X")
ax3d.plot(np.array(agent_3_plot_act_p)[:,0], np.array(agent_3_plot_act_p)[:,1], np.array(agent_3_plot_act_p)[:,2], color=color_est[2], linestyle="--")
# ax3d.plot(np.array(agent_3_plot_act_p)[:,0], np.array(agent_3_plot_act_p)[:,1], np.array(agent_3_plot_act_p)[:,2], color=color_est[2], label=r'Vehicle $f_3$ - Ground Truth', linestyle="--")
ax3d.plot(np.array(agent_3_plot_act_p)[0,0], np.array(agent_3_plot_act_p)[0,1], np.array(agent_3_plot_act_p)[0,2], color=color_est[2], ms=ms_size, marker="^", fillstyle="none")
ax3d.plot(np.array(agent_3_plot_act_p)[-1,0], np.array(agent_3_plot_act_p)[-1,1], np.array(agent_3_plot_act_p)[-1,2], color=color_est[2], ms=ms_size, marker="X", fillstyle="none")

ax3d.plot(np.array(agent_4_plot_est_p)[:,0],  np.array(agent_4_plot_est_p)[:,1],  np.array(agent_4_plot_est_p)[:,2], color=color_est[3], label=r'Vehicle $f_4$')
ax3d.plot(np.array(agent_4_plot_est_p)[0,0],  np.array(agent_4_plot_est_p)[0,1],  np.array(agent_4_plot_est_p)[0,2], color=color_est[3], ms=ms_size, marker="^")
ax3d.plot(np.array(agent_4_plot_est_p)[-1,0], np.array(agent_4_plot_est_p)[-1,1], np.array(agent_4_plot_est_p)[-1,2], color=color_est[3], ms=ms_size, marker="X")
ax3d.plot(np.array(agent_4_plot_act_p)[:,0],  np.array(agent_4_plot_act_p)[:,1],  np.array(agent_4_plot_act_p)[:,2], color=color_est[3], linestyle="--")
# ax3d.plot(np.array(agent_4_plot_act_p)[:,0],  np.array(agent_4_plot_act_p)[:,1],  np.array(agent_4_plot_act_p)[:,2], color=color_est[3], label=r'Vehicle $f_4$ - Ground Truth', linestyle="--")
ax3d.plot(np.array(agent_4_plot_act_p)[0,0],  np.array(agent_4_plot_act_p)[0,1],  np.array(agent_4_plot_act_p)[0,2], color=color_est[3], ms=ms_size, marker="^", fillstyle="none")
ax3d.plot(np.array(agent_4_plot_act_p)[-1,0], np.array(agent_4_plot_act_p)[-1,1], np.array(agent_4_plot_act_p)[-1,2], color=color_est[3], ms=ms_size, marker="X", fillstyle="none")

ax3d.plot(np.array(agent_5_plot_est_p)[:,0],  np.array(agent_5_plot_est_p)[:,1],  np.array(agent_5_plot_est_p)[:,2], color=color_est[4], label=r'Vehicle $f_5$')
ax3d.plot(np.array(agent_5_plot_est_p)[0,0],  np.array(agent_5_plot_est_p)[0,1],  np.array(agent_5_plot_est_p)[0,2], color=color_est[4], ms=ms_size, marker="^")
ax3d.plot(np.array(agent_5_plot_est_p)[-1,0], np.array(agent_5_plot_est_p)[-1,1], np.array(agent_5_plot_est_p)[-1,2], color=color_est[4], ms=ms_size, marker="X")
ax3d.plot(np.array(agent_5_plot_act_p)[:,0],  np.array(agent_5_plot_act_p)[:,1],  np.array(agent_5_plot_act_p)[:,2], color=color_est[4], linestyle="--")
# ax3d.plot(np.array(agent_5_plot_act_p)[:,0],  np.array(agent_5_plot_act_p)[:,1],  np.array(agent_5_plot_act_p)[:,2], color=color_est[4], label=r'Vehicle $f_5$ - Ground Truth', linestyle="--")
ax3d.plot(np.array(agent_5_plot_act_p)[0,0],  np.array(agent_5_plot_act_p)[0,1],  np.array(agent_5_plot_act_p)[0,2], color=color_est[4], ms=ms_size, marker="^", fillstyle="none")
ax3d.plot(np.array(agent_5_plot_act_p)[-1,0], np.array(agent_5_plot_act_p)[-1,1], np.array(agent_5_plot_act_p)[-1,2], color=color_est[4], ms=ms_size, marker="X", fillstyle="none")


landmark_size = 150
ax3d.scatter(-4, 5, 3 , color='darkblue',  s=landmark_size, marker="o")
ax3d.scatter(4, 4, 5, color='darkblue', s=landmark_size, marker="o")
ax3d.scatter(4, -3, 4, color='darkblue',  s=landmark_size, marker="o")
ax3d.legend(markerscale=25, loc="upper left", fontsize=20)
ax3d.set_xlabel("x (m)", fontsize=18)
ax3d.set_ylabel("y (m)", fontsize=18)
ax3d.set_zlabel("z (m)", fontsize=18)
ax3d.tick_params(labelsize=18)